In [1]:
import requests
import numpy as np
import unicodedata
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from shapely.geometry import Point
import cartopy.feature as cfeature
from geopy.geocoders import Nominatim
from pysus.ftp.databases.sih import SIH
from geopy.extra.rate_limiter import RateLimiter

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import threading
import os

# Arquivo "Internações limpo"

In [2]:
# cidade_codigos = {
#     "PELOTAS": 431440, "NOVO HAMBURGO": 431340, "PORTO ALEGRE": 431490, "NOVA PRATA": 431330, "VACARIA": 432250, "SÃO MARCOS": 431900, "CANOAS": 430460, 
#     "VIAMÃO": 432300, "SANTA ROSA": 431720, "BOQUEIRÃO DO LEÃO": 430245, "GARIBALDI": 430860, "TEUTÔNIA": 432145, "ALVORADA": 430060, "CACHOEIRINHA": 430310, 
#     "SAPIRANGA": 431990, "TAQUARA": 432120, "CACHOEIRA DO SUL": 430300, "GRAMADO": 430910, "ENCANTADO": 430680, "CARAZINHO": 430470, "LAJEADO": 431140, 
#     "ERECHIM": 430700, "CERRO LARGO": 430520, "SÃO LUIZ GONZAGA": 431890, "SANTO ÂNGELO": 431750, "GUARANI DAS MISSÕES": 430950, "ROQUE GONZALES": 431630, 
#     "RIO PARDO": 431570, "PORTO XAVIER": 431510, "VALE DO SOL": 432253, "VERA CRUZ": 432270, "CÂNDIDO GODÓI": 430430, "BOA VISTA DO BURICÁ": 430220, 
#     "SÃO MIGUEL DAS MISSÕES": 431915, "CAMPINA DAS MISSÕES": 430370, "PORTO LUCENA": 431500, "SANTO CRISTO": 431790, "SÃO PAULO DAS MISSÕES": 431930, 
#     "CAIBATÉ": 430330, "NOVA BRÉSCIA": 431300, "IGREJINHA": 431010, "TRIUNFO": 432200, "PAROBÉ": 431405, "GUAÍBA": 430930, "SÃO FRANCISCO DE PAULA": 431820, 
#     "SÃO SEBASTIÃO DO CAÍ": 431950, "RIOZINHO": 431575, "SALVADOR DO SUL": 431650, "CAXIAS DO SUL": 430510, "GUAPORÉ": 430940, "TAQUARI": 432130, 
#     "BENTO GONÇALVES": 430210, "VERANÓPOLIS": 432280, "SANTA CRUZ DO SUL": 431680, "SÃO JERÔNIMO": 431840, "CANDELÁRIA": 430420, "MARQUES DE SOUZA": 431205, 
#     "MUÇUM": 431260, "SINIMBU": 432067, "BOM JESUS": 430230, "SANT'ANA DO LIVRAMENTO": 431710, "ANTA GORDA": 430070, "NOVA BASSANO": 431290, "NOVA PETRÓPOLIS": 431320, 
#     "PANAMBI": 431390, "RESTINGA SECA": 431550, "SAPUCAIA DO SUL": 432000, "FELIZ": 430810, "DOIS LAJEADOS": 430645, "GUABIJU": 430925, "ITAQUI": 431060, 
#     "NONOAI": 431270, "ARATIBA": 430090, "GETÚLIO VARGAS": 430890, "CAMPINAS DO SUL": 430380, "VIADUTOS": 432290, "GAURAMA": 430870, "MARCELINO RAMOS": 431190, 
#     "ENCRUZILHADA DO SUL": 430690, "SOBRADINHO": 432070, "CRUZ ALTA": 430610, "SALTO DO JACUÍ": 431645, "PLANALTO": 431470, "CRUZEIRO DO SUL": 430620, 
#     "SÃO JOSÉ DO NORTE": 431850, "DOIS IRMÃOS": 430640, "DOM FELICIANO": 430650, "CHARQUEADAS": 430535, "TENENTE PORTELA": 432140, "CAIÇARA": 430340, 
#     "CARLOS BARBOSA": 430480, "PASSO FUNDO": 431410, "CANGUÇU": 430450, "RIO GRANDE": 431560, "SANTA MARIA": 431690, "SANTIAGO": 431740, "SANANDUVA": 431660, 
#     "MARAU": 431180, "IJUÍ": 431020, "RONDA ALTA": 431610, "URUGUAIANA": 432240, "SÃO GABRIEL": 431830, "ROSÁRIO DO SUL": 431640, "ALEGRETE": 430040, 
#     "ARVOREZINHA": 430140, "TRÊS COROAS": 432170, "CAMAQUÃ": 430350, "SANTO AUGUSTO": 431780, "BAGÉ": 430160, "DOM PEDRITO": 430660, "SANTA BÁRBARA DO SUL": 431670, 
#     "TORRES": 432150, "JÚLIO DE CASTILHOS": 431120, "FAXINAL DO SOTURNO": 430800, "PEDRO OSÓRIO": 431420, "JAGUARÃO": 431100, "FREDERICO WESTPHALEN": 430850, 
#     "RODEIO BONITO": 431590, "SEVERIANO DE ALMEIDA": 432060, "TRAMANDAÍ": 432160, "HUMAITÁ": 430970, "ESTÂNCIA VELHA": 430760, "TUPARENDI": 432230, 
#     "AUGUSTO PESTANA": 430150, "TRÊS DE MAIO": 432180, "SANTO ANTÔNIO DA PATRULHA": 431760, "MOSTARDAS": 431250, "FLORES DA CUNHA": 430820, "GIRUÁ": 430900, 
#     "CAMBARÁ DO SUL": 430360, "ARROIO GRANDE": 430130, "AGUDO": 430010, "SÃO VICENTE DO SUL": 431980, "SÃO LEOPOLDO": 431870, "SANTA VITÓRIA DO PALMAR": 431730, 
#     "JAGUARI": 431110, "HERVAL": 430710, "SÃO PEDRO DO SUL": 431940, "SÃO SEPÉ": 431960, "SÃO FRANCISCO DE ASSIS": 431810, "CACEQUI": 430290, "TAPEJARA": 432090, 
#     "TUPANCIRETÃ": 432220, "NOVA PALMA": 431310, "CHAPADA": 430530, "OSÓRIO": 431350, "ALECRIM": 430030, "HORIZONTINA": 430960, "CONSTANTINA": 430580, 
#     "SARANDI": 432010, "TRINDADE DO SUL": 432195, "PALMEIRA DAS MISSÕES": 431370, "CRISSIUMAL": 430600, "PALMARES DO SUL": 431365, "TUCUNDUVA": 432210, 
#     "JABOTICABA": 431085, "CONDOR": 430570, "CAMPO NOVO": 430400, "AJURICABA": 430020, "SÃO MARTINHO": 431910, "CORONEL BICACO": 430590, "PARAÍSO DO SUL": 431402, 
#     "SÃO JOÃO DO POLÊSINE": 431843, "SERAFINA CORRÊA": 432040, "BOM RETIRO DO SUL": 430240, "PUTINGA": 431520, "CAMPO BOM": 430390, "CANELA": 430440, 
#     "ROCA SALES": 431580, "MAXIMILIANO DE ALMEIDA": 431220, "LAGOA VERMELHA": 431130, "SEGREDO": 432026, "IBIRUBÁ": 431000, "PAIM FILHO": 431360, 
#     "CAÇAPAVA DO SUL": 430280, "CASCA": 430490, "SOLEDADE": 432080, "CAPÃO DA CANOA": 430463, "PALMITINHO": 431380, "IRAÍ": 431050, "IVOTI": 431080, 
#     "CHIAPETTA": 430540, "PEJUÇARA": 431430, "ARROIO DO TIGRE": 430120, "SÃO JOSÉ DO OURO": 431860, "RONDINHA": 431620, "SERTÃO": 432050, "ALPESTRE": 430050, 
#     "AMETISTA DO SUL": 430064, "SELBACH": 432030, "QUINZE DE NOVEMBRO": 431535, "SALDANHA MARINHO": 431643, "PINHAL GRANDE": 431447, "FORMIGUEIRO": 430840, 
#     "PARAÍ": 431400, "ACEGUÁ": 430003, "LAVRAS DO SUL": 431150, "DAVID CANABARRO": 430630, "PINHEIRO MACHADO": 431450, "ESTRELA": 430780, "GRAVATAÍ": 430920, 
#     "QUARAÍ": 431530, "VENÂNCIO AIRES": 432260, "FARROUPILHA": 430790, "ARROIO DO MEIO": 430100, "SEBERI": 432020, "TRÊS PASSOS": 432190, "SÃO LOURENÇO DO SUL": 431880, 
#     "PIRATINI": 431460, "ESTEIO": 430770, "TAPERA": 432100, "NÃO-ME-TOQUE": 431265, "PORTÃO": 431480, "ESPUMOSO": 430750, "ROLANTE": 431600, "MONTENEGRO": 431240, 
#     "ILÓPOLIS": 431030, "SÃO BORJA": 431800, "IVORÁ": 431075, "MATA": 431210, "JAQUIRANA": 431112, "ANTÔNIO PRADO": 430080, "BOM PRINCÍPIO": 430235, "PROGRESSO": 431515
# }


# anos = [str(ano) for ano in range(2008, 2024)]

# # Função para carregar dados do SIH com base na cidade e no ano
# def carregar_dados(cidade, ano):
#     sih = SIH().load()
#     # Obtendo os arquivos com base na cidade e ano
#     files = sih.get_files(cidade, uf="RS", year=ano)
#     parquets = sih.download(files)
#     data = parquets[8].to_dataframe()  # Supondo que o índice 8 seja o desejado
#     return data

# # Inicializando o aplicativo Dash com JupyterDash
# app = Dash(__name__)

# # Layout da aplicação
# app.layout = html.Div([
#     html.H1("Selecione uma Cidade e um Ano:"),
    
#     # Dropdown para seleção da cidade
#     dcc.Dropdown(
#         id='cidade-dropdown',
#         options=[{'label': cidade, 'value': cidade} for cidade in cidades],
#         value='PELOTAS',  # Valor inicial selecionado
#         style={'width': '50%'}
#     ),
    
#     # Dropdown para seleção do ano
#     dcc.Dropdown(
#         id='ano-dropdown',
#         options=[{'label': ano, 'value': ano} for ano in anos],
#         value='2023',  # Valor inicial selecionado
#         style={'width': '50%'}
#     ),
    
#     html.Div(id='cidade-selecionada'),
#     html.Div(id='ano-selecionado'),
#     html.Div(id='tabela-dados')
# ])

# # Callback para atualizar a cidade, o ano e carregar os dados
# @app.callback(
#     [Output('cidade-selecionada', 'children'),
#      Output('ano-selecionado', 'children'),
#      Output('tabela-dados', 'children')],
#     [Input('cidade-dropdown', 'value'),
#      Input('ano-dropdown', 'value')]
# )
# def update_output(cidade, ano):
#     cidade_selecionada = f'Você selecionou a cidade: {cidade}'
#     ano_selecionado = f'Você selecionou o ano: {ano}'
    
#     # Carregar os dados com base na cidade e ano selecionados
#     data = carregar_dados(cidade, ano)
    
#     # Exibir os primeiros 5 registros da tabela como exemplo
#     tabela = html.Table([
#         html.Tr([html.Th(col) for col in data.columns])  # Cabeçalho da tabela
#     ] + [
#         html.Tr([html.Td(data.iloc[i][col]) for col in data.columns])
#         for i in range(min(len(data), 5))  # Limitar aos primeiros 5 registros
#     ])
    
#     return cidade_selecionada, ano_selecionado, tabela

# # Função para rodar o Dash em uma thread separada
# def run_dash():
#     app.run_server(debug=True, use_reloader=False)

# # Rodar o Dash em uma thread separada
# thread = threading.Thread(target=run_dash)
# thread.start()


### chamada da GUI dia 29

In [3]:
# cidade_codigos = {
#     "PELOTAS": "431440", "NOVO HAMBURGO": "431340", "PORTO ALEGRE": "431490", "NOVA PRATA": "431330", "VACARIA": "432250", "SÃO MARCOS": "431900", 
#     "CANOAS": "430460", "VIAMÃO": "432300", "SANTA ROSA": "431720", "BOQUEIRÃO DO LEÃO": "430245", # adicione o restante dos códigos conforme necessário
# }



# cidade_codigos = {
#     "PELOTAS": 431440, "NOVO HAMBURGO": 431340, "PORTO ALEGRE": 431490, "NOVA PRATA": 431330, "VACARIA": 432250, "SÃO MARCOS": 431900, "CANOAS": 430460, 
#     "VIAMÃO": 432300, "SANTA ROSA": 431720, "BOQUEIRÃO DO LEÃO": 430245, "GARIBALDI": 430860, "TEUTÔNIA": 432145, "ALVORADA": 430060, "CACHOEIRINHA": 430310, 
#     "SAPIRANGA": 431990, "TAQUARA": 432120, "CACHOEIRA DO SUL": 430300, "GRAMADO": 430910, "ENCANTADO": 430680, "CARAZINHO": 430470, "LAJEADO": 431140, 
#     "ERECHIM": 430700, "CERRO LARGO": 430520, "SÃO LUIZ GONZAGA": 431890, "SANTO ÂNGELO": 431750, "GUARANI DAS MISSÕES": 430950, "ROQUE GONZALES": 431630, 
#     "RIO PARDO": 431570, "PORTO XAVIER": 431510, "VALE DO SOL": 432253, "VERA CRUZ": 432270, "CÂNDIDO GODÓI": 430430, "BOA VISTA DO BURICÁ": 430220, 
#     "SÃO MIGUEL DAS MISSÕES": 431915, "CAMPINA DAS MISSÕES": 430370, "PORTO LUCENA": 431500, "SANTO CRISTO": 431790, "SÃO PAULO DAS MISSÕES": 431930, 
#     "CAIBATÉ": 430330, "NOVA BRÉSCIA": 431300, "IGREJINHA": 431010, "TRIUNFO": 432200, "PAROBÉ": 431405, "GUAÍBA": 430930, "SÃO FRANCISCO DE PAULA": 431820, 
#     "SÃO SEBASTIÃO DO CAÍ": 431950, "RIOZINHO": 431575, "SALVADOR DO SUL": 431650, "CAXIAS DO SUL": 430510, "GUAPORÉ": 430940, "TAQUARI": 432130, 
#     "BENTO GONÇALVES": 430210, "VERANÓPOLIS": 432280, "SANTA CRUZ DO SUL": 431680, "SÃO JERÔNIMO": 431840, "CANDELÁRIA": 430420, "MARQUES DE SOUZA": 431205, 
#     "MUÇUM": 431260, "SINIMBU": 432067, "BOM JESUS": 430230, "SANT'ANA DO LIVRAMENTO": 431710, "ANTA GORDA": 430070, "NOVA BASSANO": 431290, "NOVA PETRÓPOLIS": 431320, 
#     "PANAMBI": 431390, "RESTINGA SECA": 431550, "SAPUCAIA DO SUL": 432000, "FELIZ": 430810, "DOIS LAJEADOS": 430645, "GUABIJU": 430925, "ITAQUI": 431060, 
#     "NONOAI": 431270, "ARATIBA": 430090, "GETÚLIO VARGAS": 430890, "CAMPINAS DO SUL": 430380, "VIADUTOS": 432290, "GAURAMA": 430870, "MARCELINO RAMOS": 431190, 
#     "ENCRUZILHADA DO SUL": 430690, "SOBRADINHO": 432070, "CRUZ ALTA": 430610, "SALTO DO JACUÍ": 431645, "PLANALTO": 431470, "CRUZEIRO DO SUL": 430620, 
#     "SÃO JOSÉ DO NORTE": 431850, "DOIS IRMÃOS": 430640, "DOM FELICIANO": 430650, "CHARQUEADAS": 430535, "TENENTE PORTELA": 432140, "CAIÇARA": 430340, 
#     "CARLOS BARBOSA": 430480, "PASSO FUNDO": 431410, "CANGUÇU": 430450, "RIO GRANDE": 431560, "SANTA MARIA": 431690, "SANTIAGO": 431740, "SANANDUVA": 431660, 
#     "MARAU": 431180, "IJUÍ": 431020, "RONDA ALTA": 431610, "URUGUAIANA": 432240, "SÃO GABRIEL": 431830, "ROSÁRIO DO SUL": 431640, "ALEGRETE": 430040, 
#     "ARVOREZINHA": 430140, "TRÊS COROAS": 432170, "CAMAQUÃ": 430350, "SANTO AUGUSTO": 431780, "BAGÉ": 430160, "DOM PEDRITO": 430660, "SANTA BÁRBARA DO SUL": 431670, 
#     "TORRES": 432150, "JÚLIO DE CASTILHOS": 431120, "FAXINAL DO SOTURNO": 430800, "PEDRO OSÓRIO": 431420, "JAGUARÃO": 431100, "FREDERICO WESTPHALEN": 430850, 
#     "RODEIO BONITO": 431590, "SEVERIANO DE ALMEIDA": 432060, "TRAMANDAÍ": 432160, "HUMAITÁ": 430970, "ESTÂNCIA VELHA": 430760, "TUPARENDI": 432230, 
#     "AUGUSTO PESTANA": 430150, "TRÊS DE MAIO": 432180, "SANTO ANTÔNIO DA PATRULHA": 431760, "MOSTARDAS": 431250, "FLORES DA CUNHA": 430820, "GIRUÁ": 430900, 
#     "CAMBARÁ DO SUL": 430360, "ARROIO GRANDE": 430130, "AGUDO": 430010, "SÃO VICENTE DO SUL": 431980, "SÃO LEOPOLDO": 431870, "SANTA VITÓRIA DO PALMAR": 431730, 
#     "JAGUARI": 431110, "HERVAL": 430710, "SÃO PEDRO DO SUL": 431940, "SÃO SEPÉ": 431960, "SÃO FRANCISCO DE ASSIS": 431810, "CACEQUI": 430290, "TAPEJARA": 432090, 
#     "TUPANCIRETÃ": 432220, "NOVA PALMA": 431310, "CHAPADA": 430530, "OSÓRIO": 431350, "ALECRIM": 430030, "HORIZONTINA": 430960, "CONSTANTINA": 430580, 
#     "SARANDI": 432010, "TRINDADE DO SUL": 432195, "PALMEIRA DAS MISSÕES": 431370, "CRISSIUMAL": 430600, "PALMARES DO SUL": 431365, "TUCUNDUVA": 432210, 
#     "JABOTICABA": 431085, "CONDOR": 430570, "CAMPO NOVO": 430400, "AJURICABA": 430020, "SÃO MARTINHO": 431910, "CORONEL BICACO": 430590, "PARAÍSO DO SUL": 431402, 
#     "SÃO JOÃO DO POLÊSINE": 431843, "SERAFINA CORRÊA": 432040, "BOM RETIRO DO SUL": 430240, "PUTINGA": 431520, "CAMPO BOM": 430390, "CANELA": 430440, 
#     "ROCA SALES": 431580, "MAXIMILIANO DE ALMEIDA": 431220, "LAGOA VERMELHA": 431130, "SEGREDO": 432026, "IBIRUBÁ": 431000, "PAIM FILHO": 431360, 
#     "CAÇAPAVA DO SUL": 430280, "CASCA": 430490, "SOLEDADE": 432080, "CAPÃO DA CANOA": 430463, "PALMITINHO": 431380, "IRAÍ": 431050, "IVOTI": 431080, 
#     "CHIAPETTA": 430540, "PEJUÇARA": 431430, "ARROIO DO TIGRE": 430120, "SÃO JOSÉ DO OURO": 431860, "RONDINHA": 431620, "SERTÃO": 432050, "ALPESTRE": 430050, 
#     "AMETISTA DO SUL": 430064, "SELBACH": 432030, "QUINZE DE NOVEMBRO": 431535, "SALDANHA MARINHO": 431643, "PINHAL GRANDE": 431447, "FORMIGUEIRO": 430840, 
#     "PARAÍ": 431400, "ACEGUÁ": 430003, "LAVRAS DO SUL": 431150, "DAVID CANABARRO": 430630, "PINHEIRO MACHADO": 431450, "ESTRELA": 430780, "GRAVATAÍ": 430920, 
#     "QUARAÍ": 431530, "VENÂNCIO AIRES": 432260, "FARROUPILHA": 430790, "ARROIO DO MEIO": 430100, "SEBERI": 432020, "TRÊS PASSOS": 432190, "SÃO LOURENÇO DO SUL": 431880, 
#     "PIRATINI": 431460, "ESTEIO": 430770, "TAPERA": 432100, "NÃO-ME-TOQUE": 431265, "PORTÃO": 431480, "ESPUMOSO": 430750, "ROLANTE": 431600, "MONTENEGRO": 431240, 
#     "ILÓPOLIS": 431030, "SÃO BORJA": 431800, "IVORÁ": 431075, "MATA": 431210, "JAQUIRANA": 431112, "ANTÔNIO PRADO": 430080, "BOM PRINCÍPIO": 430235, "PROGRESSO": 431515
# }


# # # Função para carregar os dados
# # def carregar_dados(cidade='PORTO ALEGRE', ano='2023'):
# #     # Validar se o ano está dentro do intervalo permitido
# #     anos_validos = list(range(2008, 2025))  # De 2008 a 2024
# #     if int(ano) not in anos_validos:
# #         raise ValueError(f"O ano {ano} não está dentro do intervalo permitido (2008-2024).")
    
# #     # Carregar o SIH
# #     sih = SIH().load()

# #     # Verificar se o código da cidade está no mapeamento
# #     codigo_cidade = cidade_codigos.get(cidade)
# #     if codigo_cidade is None:
# #         raise ValueError(f"Cidade {cidade} não encontrada no mapeamento.")
    
# #     # Obter os arquivos para o ano e cidade selecionados
# #     files = sih.get_files("SP", uf="RS", year=ano)
    
# #     # Fazer o download dos arquivos Parquet
# #     parquets = sih.download(files)
    
# #     # Verificar quantos arquivos Parquet foram baixados
# #     num_parquets = len(parquets)
# #     if num_parquets == 0:
# #         raise ValueError(f"Nenhum arquivo Parquet encontrado para o ano {ano}.")
    
# #     # Carregar os dados de todos os arquivos Parquet
# #     data = pd.concat([f.to_dataframe() for f in parquets], ignore_index=True)

# #     # Filtrar os dados pela cidade (código da cidade)
# #     data_filtrada = data[data['SP_M_HOSP'] == str(codigo_cidade)]  # Usar string para a comparação
    
# #     return data_filtrada


# from tqdm import tqdm  # Importar tqdm para mostrar barra de progresso

# # Função para carregar os dados
# def carregar_dados(cidade='PORTO ALEGRE', ano='2023'):
#     # Validar se o ano está dentro do intervalo permitido
#     anos_validos = list(range(2008, 2025))  # De 2008 a 2024
#     if int(ano) not in anos_validos:
#         raise ValueError(f"O ano {ano} não está dentro do intervalo permitido (2008-2024).")
    
#     # Carregar o SIH
#     sih = SIH().load()

#     # Verificar se o código da cidade está no mapeamento
#     codigo_cidade = cidade_codigos.get(cidade)
#     if codigo_cidade is None:
#         raise ValueError(f"Cidade {cidade} não encontrada no mapeamento.")
    
#     # Obter os arquivos para o ano e cidade selecionados
#     files = sih.get_files("SP", uf="RS", year=ano)
    
#     # Fazer o download dos arquivos Parquet
#     print(f"Iniciando o download dos arquivos Parquet para o ano {ano}...")
#     parquets = []
#     for file in tqdm(files, desc="Baixando Parquets", unit="arquivo"):
#         parquets.append(sih.download([file]))  # Baixa cada arquivo e adiciona à lista
#     print(f"Download dos arquivos concluído.")
    
#     # Verificar quantos arquivos Parquet foram baixados
#     num_parquets = len(parquets)
#     if num_parquets == 0:
#         raise ValueError(f"Nenhum arquivo Parquet encontrado para o ano {ano}.")
    
#     # Carregar os dados de todos os arquivos Parquet
#     print("Carregando os dados dos arquivos Parquet...")
#     data = pd.concat([f.to_dataframe() for f in parquets], ignore_index=True)
#     print("Dados carregados com sucesso.")
    
#     # Filtrar os dados pela cidade (código da cidade)
#     print(f"Filtrando os dados para a cidade {cidade}...")
#     data_filtrada = data[data['SP_M_HOSP'] == str(codigo_cidade)]  # Usar string para a comparação
    
#     print(f"Filtragem concluída. Total de registros encontrados: {len(data_filtrada)}")
    
#     return data_filtrada



# from dash import Dash, dcc, html, Input, Output
# import pandas as pd

# app = Dash(__name__)

# # Layout da interface gráfica
# app.layout = html.Div([
#     # Dropdown para escolher a cidade
#     dcc.Dropdown(
#         id='cidade-dropdown',
#         options=[{'label': cidade, 'value': cidade} for cidade in cidade_codigos.keys()],
#         value='PORTO ALEGRE',  # valor inicial
#     ),
#     # Dropdown para escolher o ano
#     dcc.Dropdown(
#         id='ano-dropdown',
#         options=[{'label': str(ano), 'value': str(ano)} for ano in range(2008, 2025)],
#         value='2023',  # valor inicial
#     ),
#     # Botão para iniciar o download
#     html.Button('INICIAR', id='iniciar-btn', n_clicks=0),
#     # Exibir dados ou mensagens
#     html.Div(id='output-data')
# ])

# @app.callback(
#     Output('output-data', 'children'),
#     Input('iniciar-btn', 'n_clicks'),
#     Input('cidade-dropdown', 'value'),
#     Input('ano-dropdown', 'value'),
# )
# def iniciar_download(n_clicks, cidade, ano):
#     if n_clicks > 0:
#         try:
#             # Carregar os dados após o clique no botão
#             data_filtrada = carregar_dados(cidade, ano)
            
#             # Mostrar as primeiras linhas dos dados carregados
#             return html.Div([
#                 html.H5(f'Dados carregados para {cidade} no ano {ano}'),
#                 html.Pre(data_filtrada.head().to_string())  # Exemplo de visualização dos dados
#             ])
#         except Exception as e:
#             return html.Div([f'Erro: {str(e)}'])
#     else:
#         return html.Div(['Clique no botão "INICIAR" para começar.'])

# # Rodar o servidor
# if __name__ == '__main__':
#     app.run_server(debug=True)


## ETL

### versão antiga/manual para baixar dados do pysus

In [4]:
#sih = SIH().load()

In [5]:
#sih.metadata

In [6]:
#sih.load()

In [7]:
#sih.groups

In [8]:
#files = sih.get_files("SP", uf="RS", year=2023)

In [9]:
#parquets = sih.download(files)
#parquets

In [10]:
#data =parquets[8].to_dataframe()

### versao nova do processo de ETL

In [11]:
data = pd.read_csv("dados_filtrados.csv")
data.info()

/tmp/ipykernel_33377/1469209899.py:1: DtypeWarning: Columns (16,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("dados_filtrados.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591621 entries, 0 to 591620
Data columns (total 36 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SP_GESTOR   591621 non-null  int64  
 1   SP_UF       591621 non-null  int64  
 2   SP_AA       591621 non-null  int64  
 3   SP_MM       591621 non-null  int64  
 4   SP_CNES     591621 non-null  int64  
 5   SP_NAIH     591621 non-null  int64  
 6   SP_PROCREA  591621 non-null  int64  
 7   SP_DTINTER  591621 non-null  int64  
 8   SP_DTSAIDA  591621 non-null  int64  
 9   SP_NUM_PR   0 non-null       float64
 10  SP_TIPO     0 non-null       float64
 11  SP_CPFCGC   591621 non-null  int64  
 12  SP_ATOPROF  591621 non-null  int64  
 13  SP_TP_ATO   0 non-null       float64
 14  SP_QTD_ATO  591621 non-null  int64  
 15  SP_PTSP     591621 non-null  int64  
 16  SP_NF       12170 non-null   object 
 17  SP_VALATO   591621 non-null  float64
 18  SP_M_HOSP   591621 non-null  int64  
 19  SP

In [12]:
data_clean = data.drop(['SP_GESTOR','SP_NUM_PR', 'SP_TIPO', 'SP_TP_ATO','SP_CPFCGC', 'SP_QTD_ATO', 'SP_PTSP', 'SP_NF', 'SP_COMPLEX', 'SP_FINANC', 'SP_CO_FAEC', 'SP_PF_DOC', 'SP_PJ_DOC','IN_TP_VAL', 'SEQUENCIA', 'REMESSA', 'SERV_CLA', 'SP_CIDPRI', 'SP_CIDSEC', 'SP_U_AIH', 'SP_PF_CBO'], axis='columns')

In [13]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591621 entries, 0 to 591620
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SP_UF       591621 non-null  int64  
 1   SP_AA       591621 non-null  int64  
 2   SP_MM       591621 non-null  int64  
 3   SP_CNES     591621 non-null  int64  
 4   SP_NAIH     591621 non-null  int64  
 5   SP_PROCREA  591621 non-null  int64  
 6   SP_DTINTER  591621 non-null  int64  
 7   SP_DTSAIDA  591621 non-null  int64  
 8   SP_ATOPROF  591621 non-null  int64  
 9   SP_VALATO   591621 non-null  float64
 10  SP_M_HOSP   591621 non-null  int64  
 11  SP_M_PAC    591621 non-null  int64  
 12  SP_DES_HOS  591621 non-null  int64  
 13  SP_DES_PAC  591621 non-null  int64  
 14  SP_QT_PROC  591621 non-null  int64  
dtypes: float64(1), int64(14)
memory usage: 67.7 MB


In [14]:
data_clean.head()

,SP_UF,SP_AA,SP_MM,SP_CNES,SP_NAIH,SP_PROCREA,SP_DTINTER,SP_DTSAIDA,SP_ATOPROF,SP_VALATO,SP_M_HOSP,SP_M_PAC,SP_DES_HOS,SP_DES_PAC,SP_QT_PROC
0,43,2022,1,3508528,4321109205787,303010037,20211110,20211209,204060095,0.00,430460,430460,0,0,1
1,43,2022,1,3508528,4321109205787,303010037,20211110,20211209,211020036,0.00,430460,430460,0,0,3
2,43,2022,1,3508528,4321109205897,303040297,20211129,20211228,303040297,87.85,430460,430460,0,0,0
3,43,2022,1,3508528,4321109205897,303040297,20211129,20211228,302040013,19.05,430460,430460,0,0,3
4,43,2022,1,3508528,4321109205897,303040297,20211129,20211228,302040013,127.00,430460,430460,0,0,20


In [15]:
print("dados da coluna M HOSP: ")
print(data_clean['SP_M_HOSP'].value_counts())

dados da coluna M HOSP: 
SP_M_HOSP
430460    591621
Name: count, dtype: int64


In [16]:
arquivo = 'Nomes_cidades'
codigo_cidade = {}
with open(arquivo, 'r') as f:
    linhas = f.readlines()

for linha in linhas:
    partes = linha.split()
    if len(partes) > 2:  # Verifica se há pelo menos 3 elementos na linha
        try:
            codigo_cid = str(partes[0])  # Converte para string
            nome_cidade = ' '.join(partes[2:-1])
            codigo_cidade[codigo_cid] = nome_cidade
        except (ValueError, IndexError) as e:
            print(f"Erro ao processar a linha: {linha.strip()}")
            print(f"Erro: {e}")

# Verificação
if '430040' in codigo_cidade:
    print("A variável 430040 está presente no dicionário.")
else:
    print("A variável 430040 não está presente no dicionário.")

print(codigo_cidade)

A variável 430040 está presente no dicionário.
{'430003': 'GRANDE DO SUL ACEGUÁ', '430005': 'GRANDE DO SUL ÁGUA SANTA', '430010': 'GRANDE DO SUL AGUDO', '430020': 'GRANDE DO SUL AJURICABA', '430030': 'GRANDE DO SUL ALECRIM', '430040': 'GRANDE DO SUL ALEGRETE', '430045': 'GRANDE DO SUL ALEGRIA', '430047': 'GRANDE DO SUL ALMIRANTE TAMANDARÉ DO SUL', '430050': 'GRANDE DO SUL ALPESTRE', '430055': 'GRANDE DO SUL ALTO ALEGRE', '430057': 'GRANDE DO SUL ALTO FELIZ', '430060': 'GRANDE DO SUL ALVORADA', '430063': 'GRANDE DO SUL AMARAL FERRADOR', '430064': 'GRANDE DO SUL AMETISTA DO SUL', '430066': 'GRANDE DO SUL ANDRÉ DA ROCHA', '430070': 'GRANDE DO SUL ANTA GORDA', '430080': 'GRANDE DO SUL ANTÔNIO PRADO', '430085': 'GRANDE DO SUL ARAMBARÉ', '430087': 'GRANDE DO SUL ARARICÁ', '430090': 'GRANDE DO SUL ARATIBA', '430100': 'GRANDE DO SUL ARROIO DO MEIO', '430105': 'GRANDE DO SUL ARROIO DO SAL', '430107': 'GRANDE DO SUL ARROIO DO PADRE', '430110': 'GRANDE DO SUL ARROIO DOS RATOS', '430120': 'GRANDE 

In [17]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591621 entries, 0 to 591620
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SP_UF       591621 non-null  int64  
 1   SP_AA       591621 non-null  int64  
 2   SP_MM       591621 non-null  int64  
 3   SP_CNES     591621 non-null  int64  
 4   SP_NAIH     591621 non-null  int64  
 5   SP_PROCREA  591621 non-null  int64  
 6   SP_DTINTER  591621 non-null  int64  
 7   SP_DTSAIDA  591621 non-null  int64  
 8   SP_ATOPROF  591621 non-null  int64  
 9   SP_VALATO   591621 non-null  float64
 10  SP_M_HOSP   591621 non-null  int64  
 11  SP_M_PAC    591621 non-null  int64  
 12  SP_DES_HOS  591621 non-null  int64  
 13  SP_DES_PAC  591621 non-null  int64  
 14  SP_QT_PROC  591621 non-null  int64  
dtypes: float64(1), int64(14)
memory usage: 67.7 MB


In [18]:
data_clean['SP_CIDADE_H'] = data_clean['SP_M_HOSP'].astype(str).map(codigo_cidade)
print(data_clean)

        SP_UF  SP_AA  SP_MM  SP_CNES        SP_NAIH  SP_PROCREA  SP_DTINTER  \
0          43   2022      1  3508528  4321109205787   303010037    20211110   
1          43   2022      1  3508528  4321109205787   303010037    20211110   
2          43   2022      1  3508528  4321109205897   303040297    20211129   
3          43   2022      1  3508528  4321109205897   303040297    20211129   
4          43   2022      1  3508528  4321109205897   303040297    20211129   
...       ...    ...    ...      ...            ...         ...         ...   
591616     43   2022     12  2232014  4322108871883   407020039    20221204   
591617     43   2022     12  2232014  4322108871883   407020039    20221204   
591618     43   2022     12  2232014  4322108871883   407020039    20221204   
591619     43   2022     12  2232014  4322108871949   409010170    20221209   
591620     43   2022     12  2232014  4322108871949   409010170    20221209   

        SP_DTSAIDA  SP_ATOPROF  SP_VALATO  SP_M_HOS

In [19]:
print("dados da coluna CIDADE H: ")
print(data_clean['SP_CIDADE_H'].value_counts())
print("dados da coluna M HOSP: ")
print(data_clean['SP_M_HOSP'].value_counts())

dados da coluna CIDADE H: 
SP_CIDADE_H
GRANDE DO SUL CANOAS    591621
Name: count, dtype: int64
dados da coluna M HOSP: 
SP_M_HOSP
430460    591621
Name: count, dtype: int64


In [20]:
data_clean['SP_CIDADE_P'] = data_clean['SP_M_PAC'].astype(str).map(codigo_cidade)
print(data_clean)

        SP_UF  SP_AA  SP_MM  SP_CNES        SP_NAIH  SP_PROCREA  SP_DTINTER  \
0          43   2022      1  3508528  4321109205787   303010037    20211110   
1          43   2022      1  3508528  4321109205787   303010037    20211110   
2          43   2022      1  3508528  4321109205897   303040297    20211129   
3          43   2022      1  3508528  4321109205897   303040297    20211129   
4          43   2022      1  3508528  4321109205897   303040297    20211129   
...       ...    ...    ...      ...            ...         ...         ...   
591616     43   2022     12  2232014  4322108871883   407020039    20221204   
591617     43   2022     12  2232014  4322108871883   407020039    20221204   
591618     43   2022     12  2232014  4322108871883   407020039    20221204   
591619     43   2022     12  2232014  4322108871949   409010170    20221209   
591620     43   2022     12  2232014  4322108871949   409010170    20221209   

        SP_DTSAIDA  SP_ATOPROF  SP_VALATO  SP_M_HOS

In [21]:
# String GRANDE DO SUL precisa ser retirada do dataset
rs = 'GRANDE DO SUL'
data_clean['SP_CIDADE_H'] = data_clean['SP_CIDADE_H'].str.replace(rs, '').str.strip()
data_clean['SP_CIDADE_P'] = data_clean['SP_CIDADE_P'].str.replace(rs, '').str.strip()

print(data_clean)

        SP_UF  SP_AA  SP_MM  SP_CNES        SP_NAIH  SP_PROCREA  SP_DTINTER  \
0          43   2022      1  3508528  4321109205787   303010037    20211110   
1          43   2022      1  3508528  4321109205787   303010037    20211110   
2          43   2022      1  3508528  4321109205897   303040297    20211129   
3          43   2022      1  3508528  4321109205897   303040297    20211129   
4          43   2022      1  3508528  4321109205897   303040297    20211129   
...       ...    ...    ...      ...            ...         ...         ...   
591616     43   2022     12  2232014  4322108871883   407020039    20221204   
591617     43   2022     12  2232014  4322108871883   407020039    20221204   
591618     43   2022     12  2232014  4322108871883   407020039    20221204   
591619     43   2022     12  2232014  4322108871949   409010170    20221209   
591620     43   2022     12  2232014  4322108871949   409010170    20221209   

        SP_DTSAIDA  SP_ATOPROF  SP_VALATO  SP_M_HOS

In [22]:
nulos_p = data_clean['SP_CIDADE_P'].isna().sum()
nulos_h = data_clean['SP_CIDADE_H'].isna().sum()
print(f"Total de valores nulos na coluna 'SP_CIDADE_P': {nulos_p}")
print(f"Total de valores nulos na coluna 'SP_CIDADE_H': {nulos_h}")

Total de valores nulos na coluna 'SP_CIDADE_P': 1860
Total de valores nulos na coluna 'SP_CIDADE_H': 0


In [23]:
codigos_nulos = data_clean[data_clean['SP_CIDADE_P'].isnull()]['SP_M_PAC'].unique()

print("Códigos SP_M_HOSP com valores nulos na coluna SP_CIDADE_H:")
print(codigos_nulos)

Códigos SP_M_HOSP com valores nulos na coluna SP_CIDADE_H:
[410830 220795 354780 291840 330540 420205 421150 251370 355370 355630
 420880 411620 421580 412770 410180 421660 171200 320520 410690 351870
 110140 354880 310670 291670 120040 500375 120020 412480 260400 410420
 355030 330520 352940 315700 316120 411370]


In [24]:
arquivo = 'Nomes_cidades_externas'
codigo_cidade_externa = {}

with open(arquivo, 'r') as f:
    linhas = f.readlines()

for linha in linhas:
    partes = linha.split()    
    codigo_cid = partes[0]
    
    nome_cidade = ' '.join(partes[2:-1])
    codigo_cidade_externa[codigo_cid] = nome_cidade



In [25]:
def verifica_cidade_exterior(sp_m_pac):
    if sp_m_pac not in codigo_cidade:
        return codigo_cidade_externa.get(sp_m_pac, 'Cidade do RS')
    else:
        return None
data_clean['SP_CIDADE_P_EXT'] = data_clean['SP_M_PAC'].apply(lambda x: verifica_cidade_exterior(x))
print(data_clean)

        SP_UF  SP_AA  SP_MM  SP_CNES        SP_NAIH  SP_PROCREA  SP_DTINTER  \
0          43   2022      1  3508528  4321109205787   303010037    20211110   
1          43   2022      1  3508528  4321109205787   303010037    20211110   
2          43   2022      1  3508528  4321109205897   303040297    20211129   
3          43   2022      1  3508528  4321109205897   303040297    20211129   
4          43   2022      1  3508528  4321109205897   303040297    20211129   
...       ...    ...    ...      ...            ...         ...         ...   
591616     43   2022     12  2232014  4322108871883   407020039    20221204   
591617     43   2022     12  2232014  4322108871883   407020039    20221204   
591618     43   2022     12  2232014  4322108871883   407020039    20221204   
591619     43   2022     12  2232014  4322108871949   409010170    20221209   
591620     43   2022     12  2232014  4322108871949   409010170    20221209   

        SP_DTSAIDA  SP_ATOPROF  SP_VALATO  SP_M_HOS

In [26]:
substituicoes = {
    'CATARINA': '',
    'GERAIS ': '',
    'PAULO ': '',
    'DE JANEIRO ': '',
    'GROSSO DO SUL ': '',
    'GROSSO ': ''
}

data_clean['SP_CIDADE_P_EXT'] = data_clean['SP_CIDADE_P_EXT'].replace(substituicoes, regex=True).str.strip()

tipos_dados = data_clean['SP_CIDADE_P_EXT'].unique()
print(tipos_dados)

['Cidade do RS']


In [27]:
# Substituir todos os valores None por 'Cidade do RS' na coluna SP_CIDADE_P_EXT
data_clean['SP_CIDADE_P_EXT'].fillna('Cidade do RS', inplace=True)
print(data_clean)

        SP_UF  SP_AA  SP_MM  SP_CNES        SP_NAIH  SP_PROCREA  SP_DTINTER  \
0          43   2022      1  3508528  4321109205787   303010037    20211110   
1          43   2022      1  3508528  4321109205787   303010037    20211110   
2          43   2022      1  3508528  4321109205897   303040297    20211129   
3          43   2022      1  3508528  4321109205897   303040297    20211129   
4          43   2022      1  3508528  4321109205897   303040297    20211129   
...       ...    ...    ...      ...            ...         ...         ...   
591616     43   2022     12  2232014  4322108871883   407020039    20221204   
591617     43   2022     12  2232014  4322108871883   407020039    20221204   
591618     43   2022     12  2232014  4322108871883   407020039    20221204   
591619     43   2022     12  2232014  4322108871949   409010170    20221209   
591620     43   2022     12  2232014  4322108871949   409010170    20221209   

        SP_DTSAIDA  SP_ATOPROF  SP_VALATO  SP_M_HOS

/tmp/ipykernel_33377/3527762726.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_clean['SP_CIDADE_P_EXT'].fillna('Cidade do RS', inplace=True)


In [28]:
contando_dados= data_clean['SP_CIDADE_P_EXT'].value_counts()
contando_dados

SP_CIDADE_P_EXT
Cidade do RS    591621
Name: count, dtype: int64

In [29]:
data_clean['SP_CIDADE_P'].fillna('Cidade de fora do RS', inplace=True)

/tmp/ipykernel_33377/321287065.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_clean['SP_CIDADE_P'].fillna('Cidade de fora do RS', inplace=True)


In [30]:
nulos_p = data_clean['SP_CIDADE_P'].isna().sum()
nulos_ext = data_clean['SP_CIDADE_P_EXT'].isna().sum()
nulos_h = data_clean['SP_CIDADE_H'].isna().sum()
print(f"Total de valores nulos na coluna 'SP_CIDADE_P': {nulos_p}")
print(f"Total de valores nulos na coluna 'SP_CIDADE_P_EXT': {nulos_ext}")
print(f"Total de valores nulos na coluna 'SP_CIDADE_H': {nulos_h}")

Total de valores nulos na coluna 'SP_CIDADE_P': 0
Total de valores nulos na coluna 'SP_CIDADE_P_EXT': 0
Total de valores nulos na coluna 'SP_CIDADE_H': 0


In [31]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591621 entries, 0 to 591620
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            591621 non-null  int64  
 1   SP_AA            591621 non-null  int64  
 2   SP_MM            591621 non-null  int64  
 3   SP_CNES          591621 non-null  int64  
 4   SP_NAIH          591621 non-null  int64  
 5   SP_PROCREA       591621 non-null  int64  
 6   SP_DTINTER       591621 non-null  int64  
 7   SP_DTSAIDA       591621 non-null  int64  
 8   SP_ATOPROF       591621 non-null  int64  
 9   SP_VALATO        591621 non-null  float64
 10  SP_M_HOSP        591621 non-null  int64  
 11  SP_M_PAC         591621 non-null  int64  
 12  SP_DES_HOS       591621 non-null  int64  
 13  SP_DES_PAC       591621 non-null  int64  
 14  SP_QT_PROC       591621 non-null  int64  
 15  SP_CIDADE_H      591621 non-null  object 
 16  SP_CIDADE_P      591621 non-null  obje

In [32]:
arquivo = 'codigo_procedimento'
codigo_procedimento = {}

with open(arquivo, 'r') as f:
    linhas = f.readlines()

for linha in linhas:
    partes = linha.strip().split()
    codigo = partes[0]
    titulo = []

    for parte in partes[1:]:
        if any(char.isdigit() for char in parte):
            break
        titulo.append(parte)
    
    # Juntar o nome da atividade
    titulo = ' '.join(titulo)
    
    # Adicionar ao dicionário
    codigo_procedimento[codigo] = titulo

# Adicionando manualmente os códigos ausentes pegos no site http://sigtap.datasus.gov.br/tabela-unificada/app/sec/inicio.jsp
codigo_procedimento.update({
    '0308010019': 'TRATAMENTO CLÍNICO/CONSERVADOR DE TRAUMATISMOS DE QUALQUER LOCALIZAÇÃO',
    '0303170140': 'TRATAMENTO CLÍNICO PARA CONTENÇÃO DE COMPORTAMENTO DESORGANIZADO E/OU DISRUPTIVO',
    '0303170166': 'TRATAMENTO CLÍNICO DE TRANSTORNOS MENTAIS E COMPORTAMENTAIS DEVIDO AO USO DE ÁLCOOL',
    '0303170174': 'TRATAMENTO CLÍNICO DE TRANSTORNOS MENTAIS E COMPORTAMENTAIS DEVIDO AO USO DE “CRACK”',
    '0303170182': 'TRATAMENTO CLÍNICO DOS TRANSTORNOS MENTAIS E COMPORTAMENTAIS DEVIDO AO USO DAS DEMAIS DROGAS E/OU OUTRAS SUBSTÂNCIAS PSICOATIVAS'
})

#print(codigo_procedimento)

In [33]:
print("Exemplo de chave no dicionário:", next(iter(codigo_procedimento.keys())))
print("Exemplo de valor em SP_PROCREA:", data_clean['SP_PROCREA'].astype(str).iloc[0])

data_clean['SP_PROCREA'] = data_clean['SP_PROCREA'].astype(str).str.zfill(10)
data_clean['SP_ATOPROF'] = data_clean['SP_ATOPROF'].astype(str).str.zfill(10)
codigo_procedimento = {k.strip(): v.strip() for k, v in codigo_procedimento.items()}
print(set(data_clean['SP_PROCREA'].unique()) - set(codigo_procedimento.keys()))



Exemplo de chave no dicionário: 0101010028


Exemplo de valor em SP_PROCREA: 303010037


{'0413040259', '0416030343', '0416030297', '0416020232', '0416090133', '0415010012', '0415020069', '0408050888', '0416020224', '0303180013', '0415020018', '0416020178', '0416040241', '0416030270', '0416040268', '0415030013', '0415020077', '0416040217', '0415020034', '0416020151', '0416060110', '0416080120', '0415020050', '0416020259', '0503010022', '0404010032', '0416060129', '0416020208', '0408050160', '0303170131', '0404010016', '0303170158', '0416040276', '0416020216', '0403010390', '0404010024', '0416040209', '0409010596', '0408050667', '0416020194', '0416010229', '0416010210', '0404010415', '0303040300', '0303010223', '0503010014'}


In [34]:
# Converter SP_PROCREA para string e mapear
data_clean['SP_PROCREA'] = data_clean['SP_PROCREA'].astype(str)
data_clean['SP_N_PROC'] = data_clean['SP_PROCREA'].map(codigo_procedimento)

# Converter SP_ATOPROF para string e mapear
data_clean['SP_ATOPROF'] = data_clean['SP_ATOPROF'].astype(str)
data_clean['SP_N_ATOS'] = data_clean['SP_ATOPROF'].map(codigo_procedimento)

# Contar dados
contando_dados_proc = data_clean['SP_N_PROC'].value_counts()
contando_dados_prof = data_clean['SP_N_ATOS'].value_counts()

print("Contagem de procedimentos:\n", contando_dados_proc)
print("\nContagem de profissionais:\n", contando_dados_prof)

Contagem de procedimentos:


 SP_N_PROC
TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS                                           119426
TRATAMENTO DE ACIDENTE VASCULAR CEREBRAL - AVC (ISQUEMICO OU HEMORRAGICO AGUDO)     21984
PARTO NORMAL                                                                        21012
PARTO CESARIANO EM GESTACAO DE ALTO RISCO                                           19017
TRATAMENTO DE INSUFICIENCIA CARDIACA                                                15432
                                                                                    ...  
RECONSTRUCAO DE POLO SUPERIOR DA ORELHA                                                 3
REDUÇÃO DE FRATURA DA MANDÍBULA SEM OSTEOSSINTESE.                                      2
TRATAMENTO DE PTOSE PALPEBRAL                                                           2
OSTEOSSÍNTESE DE FRATURA SIMPLES DE MANDÍBULA                                           2
EXERESE DE CALAZIO E OUTRAS PEQUENAS LESOES DA PALPEBRA E SUPERCILIOS                   2

### Limpando Colunas com nulos e criando csv

In [35]:
nulos_proc = data_clean['SP_N_PROC'].isna().sum()
nulos_atos = data_clean['SP_N_ATOS'].isna().sum()
print(f"Total de valores nulos na coluna 'SP_N_PROC': {nulos_proc}")
print(f"Total de valores nulos na coluna 'SP_N_ATOS': {nulos_atos}")

print()
dados_proc = data_clean['SP_N_PROC'].notna().sum()
dados_atos = data_clean['SP_N_ATOS'].notna().sum()
print(f"Total de valores NAO nulos na coluna 'SP_N_PROC': {dados_proc}")
print(f"Total de valores NAO nulos na coluna 'SP_N_ATOS': {dados_atos}")

Total de valores nulos na coluna 'SP_N_PROC': 80954
Total de valores nulos na coluna 'SP_N_ATOS': 41447

Total de valores NAO nulos na coluna 'SP_N_PROC': 510667
Total de valores NAO nulos na coluna 'SP_N_ATOS': 550174


In [36]:
def verificar_codigos(codigos, dicionario):
    presentes = []
    ausentes = []
    for codigo in codigos:
        if codigo in dicionario:
            presentes.append(codigo)
        else:
            ausentes.append(codigo)
    return presentes, ausentes

In [37]:
nulos_proc = data_clean['SP_N_PROC'].isna().sum()
nulos_atos = data_clean['SP_N_ATOS'].isna().sum()
print(f"Total de valores nulos na coluna 'SP_N_PROC': {nulos_proc}")
print(f"Total de valores nulos na coluna 'SP_N_ATOS': {nulos_atos}")

print()
dados_proc = data_clean['SP_N_PROC'].notna().sum()
dados_atos = data_clean['SP_N_ATOS'].notna().sum()
print(f"Total de valores NAO nulos na coluna 'SP_N_PROC': {dados_proc}")
print(f"Total de valores NAO nulos na coluna 'SP_N_ATOS': {dados_atos}")

Total de valores nulos na coluna 'SP_N_PROC': 80954
Total de valores nulos na coluna 'SP_N_ATOS': 41447

Total de valores NAO nulos na coluna 'SP_N_PROC': 510667
Total de valores NAO nulos na coluna 'SP_N_ATOS': 550174


In [38]:
data_clean.dropna(subset=['SP_N_PROC'], inplace= True)
data_clean.dropna(subset=['SP_N_ATOS'], inplace= True)


In [39]:
data_limpo = data_clean.drop_duplicates()

In [40]:
data_limpo.info()
data_limpo.to_csv('dados_limpos.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 462630 entries, 0 to 591620
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            462630 non-null  int64  
 1   SP_AA            462630 non-null  int64  
 2   SP_MM            462630 non-null  int64  
 3   SP_CNES          462630 non-null  int64  
 4   SP_NAIH          462630 non-null  int64  
 5   SP_PROCREA       462630 non-null  object 
 6   SP_DTINTER       462630 non-null  int64  
 7   SP_DTSAIDA       462630 non-null  int64  
 8   SP_ATOPROF       462630 non-null  object 
 9   SP_VALATO        462630 non-null  float64
 10  SP_M_HOSP        462630 non-null  int64  
 11  SP_M_PAC         462630 non-null  int64  
 12  SP_DES_HOS       462630 non-null  int64  
 13  SP_DES_PAC       462630 non-null  int64  
 14  SP_QT_PROC       462630 non-null  int64  
 15  SP_CIDADE_H      462630 non-null  object 
 16  SP_CIDADE_P      462630 non-null  object 
 

## Nova descrição das colunas

# Revisao de cidades

In [41]:
data_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 462630 entries, 0 to 591620
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            462630 non-null  int64  
 1   SP_AA            462630 non-null  int64  
 2   SP_MM            462630 non-null  int64  
 3   SP_CNES          462630 non-null  int64  
 4   SP_NAIH          462630 non-null  int64  
 5   SP_PROCREA       462630 non-null  object 
 6   SP_DTINTER       462630 non-null  int64  
 7   SP_DTSAIDA       462630 non-null  int64  
 8   SP_ATOPROF       462630 non-null  object 
 9   SP_VALATO        462630 non-null  float64
 10  SP_M_HOSP        462630 non-null  int64  
 11  SP_M_PAC         462630 non-null  int64  
 12  SP_DES_HOS       462630 non-null  int64  
 13  SP_DES_PAC       462630 non-null  int64  
 14  SP_QT_PROC       462630 non-null  int64  
 15  SP_CIDADE_H      462630 non-null  object 
 16  SP_CIDADE_P      462630 non-null  object 
 

In [42]:
data_limpo.head()

,SP_UF,SP_AA,SP_MM,SP_CNES,SP_NAIH,SP_PROCREA,SP_DTINTER,SP_DTSAIDA,SP_ATOPROF,SP_VALATO,SP_M_HOSP,SP_M_PAC,SP_DES_HOS,SP_DES_PAC,SP_QT_PROC,SP_CIDADE_H,SP_CIDADE_P,SP_CIDADE_P_EXT,SP_N_PROC,SP_N_ATOS
0,43,2022,1,3508528,4321109205787,0303010037,20211110,20211209,0204060095,0.00,430460,430460,0,0,1,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,RADIOGRAFIA DE BACIA
1,43,2022,1,3508528,4321109205787,0303010037,20211110,20211209,0211020036,0.00,430460,430460,0,0,3,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,ELETROCARDIOGRAMA
2,43,2022,1,3508528,4321109205897,0303040297,20211129,20211228,0303040297,87.85,430460,430460,0,0,0,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...
3,43,2022,1,3508528,4321109205897,0303040297,20211129,20211228,0302040013,19.05,430460,430460,0,0,3,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,ATENDIMENTO FISIOTERAPÊUTICO EM PACIENTE C/ TR...
4,43,2022,1,3508528,4321109205897,0303040297,20211129,20211228,0302040013,127.00,430460,430460,0,0,20,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,ATENDIMENTO FISIOTERAPÊUTICO EM PACIENTE C/ TR...


In [43]:
# import pandas as pd

# municipios_presentes = [
#     "AGUDO", "AJURICABA", "ALECRIM", "ALEGRETE", "ALPESTRE", "ALVORADA", "AMETISTA DO SUL", "ANTA GORDA", 
#     "ARATIBA", "ARROIO DO TIGRE", "ARROIO GRANDE", "ARVOREZINHA", "AUGUSTO PESTANA", "BAGÉ", "BENTO GONÇALVES", 
#     "BOA VISTA DO BURICÁ", "BOM JESUS", "BOM RETIRO DO SUL", "BOQUEIRÃO DO LEÃO", "CACEQUI", "CAÇAPAVA DO SUL", 
#     "CACHOEIRA DO SUL", "CACHOEIRINHA", "CAIBATÉ", "CAIÇARA", "CAMAQUÃ", "CAMBARÁ DO SUL", "CAMPINA DAS MISSÕES", 
#     "CAMPINAS DO SUL", "CAMPO BOM", "CAMPO NOVO", "CANDELÁRIA", "CANDIDO GODÓI", "CANOAS", "CAPÃO DA CANOA", 
#     "CARAZINHO", "CARLOS BARBOSA", "CASCA", "CAXIAS DO SUL", "CERRO LARGO", "CHARQUEADAS", "CHAPADA", "CHIAPETTA", 
#     "CONSTANTINA", "CONDOR", "CORONEL BICACO", "CRISSIUMAL", "CRUZ ALTA", "CRUZEIRO DO SUL", "DOIS IRMÃOS", 
#     "DOIS LAJEADOS", "DOM FELICIANO", "DOM PEDRITO", "ENCANTADO", "ENCRUZILHADA DO SUL", "ERECHIM", "FAXINAL DO SOTURNO", 
#     "FELIZ", "FLORES DA CUNHA", "FREDERICO WESTPHALEN", "GARIBALDI", "GAURAMA", "GETÚLIO VARGAS", "GIRUÁ", "GRAMADO", 
#     "GUAÍBA", "GUABIJU", "GUAPORÉ", "GUARANI DAS MISSÕES", "HERVAL", "HORIZONTINA", "HUMAITÁ", "IBIRUBÁ", "IJUÍ", 
#     "IGREJINHA", "IRAÍ", "ITAQUI", "IVOTI", "JABOTICABA", "JAGUARÃO", "JAGUARI", "JÚLIO DE CASTILHOS", "LAGOA VERMELHA", 
#     "LAJEADO", "MARCELINO RAMOS", "MARAU", "MARQUES DE SOUZA", "MAXIMILIANO DE ALMEIDA", "MOSTARDAS", "MUÇUM", "NONOAI", 
#     "NOVA BASSANO", "NOVA BRÉSCIA", "NOVA PALMA", "NOVA PETRÓPOLIS", "NOVA PRATA", "NOVO HAMBURGO", "OSÓRIO", 
#     "PAIM FILHO", "PALMARES DO SUL", "PALMEIRA DAS MISSÕES", "PANAMBI", "PARAÍSO DO SUL", "PAROBÉ", "PASSO FUNDO", 
#     "PEJUÇARA", "PELOTAS", "PLANALTO", "PORTO ALEGRE", "PORTO LUCENA", "PORTO XAVIER", "PUTINGA", "QUINZE DE NOVEMBRO", 
#     "RESTINGA SECA", "RIO GRANDE", "RIO PARDO", "ROCA SALES", "RODEIO BONITO", "RONDINHA", "ROQUE GONZALES", 
#     "ROSÁRIO DO SUL", "SALTO DO JACUÍ", "SALVADOR DO SUL", "SANANDUVA", "SANTA BÁRBARA DO SUL", "SANTA CRUZ DO SUL", 
#     "SANTA MARIA", "SANTA ROSA", "SANTA VITÓRIA DO PALMAR", "SANTIAGO", "SANTO ÂNGELO", "SANTO AUGUSTO", "SANTO CRISTO", 
#     "SÃO FRANCISCO DE ASSIS", "SÃO GABRIEL", "SÃO JERÔNIMO", "SÃO JOÃO DO POLÊSINE", "SÃO JOSÉ DO NORTE", 
#     "SÃO JOSÉ DO OURO", "SÃO LEOPOLDO", "SÃO LUIZ GONZAGA", "SÃO MARCOS", "SÃO MIGUEL DAS MISSÕES", "SÃO PEDRO DO SUL", 
#     "SAPIRANGA", "SAPUCAIA DO SUL", "SARANDI", "SOLEDADE", "TAPEJARA", "TAQUARA", "TAQUARI", "TENENTE PORTELA", "TORRES", 
#     "TRAMANDAÍ", "TRÊS COROAS", "TRÊS DE MAIO", "TRINDADE DO SUL", "TRIUNFO", "TUCUNDUVA", "TUPANCIRETÃ", "TUPARENDI", 
#     "URUGUAIANA", "VALE DO SOL", "VACARIA", "VERA CRUZ", "VERANÓPOLIS", "VIADUTOS", "VIAMÃO"]

# # Identificando os municípios que não estão na lista
# municipios_faltantes = sorted(set(data_limpo["SP_CIDADE_H"]) - set(municipios_presentes))

# print("Municípios faltantes:", municipios_faltantes)


In [44]:
# # Criando o dicionário com os municípios faltantes e seus respectivos códigos
# cidades_faltantes = data_limpo[data_limpo["SP_CIDADE_H"].isin(municipios_faltantes)].set_index("SP_CIDADE_H")["SP_M_HOSP"].to_dict()

# print(cidades_faltantes)


In [45]:
data_limpo['SP_CIDADE_H'] = data_limpo['SP_CIDADE_H'].replace("SANT'ANA DO LIVRAMENTO", "SANTANA DO LIVRAMENTO")
data_limpo['SP_CIDADE_P'] = data_limpo['SP_CIDADE_P'].replace("SANT'ANA DO LIVRAMENTO", "SANTANA DO LIVRAMENTO")


/tmp/ipykernel_33377/1495055583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpo['SP_CIDADE_H'] = data_limpo['SP_CIDADE_H'].replace("SANT'ANA DO LIVRAMENTO", "SANTANA DO LIVRAMENTO")
/tmp/ipykernel_33377/1495055583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpo['SP_CIDADE_P'] = data_limpo['SP_CIDADE_P'].replace("SANT'ANA DO LIVRAMENTO", "SANTANA DO LIVRAMENTO")


In [46]:
data_limpo.head()

,SP_UF,SP_AA,SP_MM,SP_CNES,SP_NAIH,SP_PROCREA,SP_DTINTER,SP_DTSAIDA,SP_ATOPROF,SP_VALATO,SP_M_HOSP,SP_M_PAC,SP_DES_HOS,SP_DES_PAC,SP_QT_PROC,SP_CIDADE_H,SP_CIDADE_P,SP_CIDADE_P_EXT,SP_N_PROC,SP_N_ATOS
0,43,2022,1,3508528,4321109205787,0303010037,20211110,20211209,0204060095,0.00,430460,430460,0,0,1,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,RADIOGRAFIA DE BACIA
1,43,2022,1,3508528,4321109205787,0303010037,20211110,20211209,0211020036,0.00,430460,430460,0,0,3,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,ELETROCARDIOGRAMA
2,43,2022,1,3508528,4321109205897,0303040297,20211129,20211228,0303040297,87.85,430460,430460,0,0,0,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...
3,43,2022,1,3508528,4321109205897,0303040297,20211129,20211228,0302040013,19.05,430460,430460,0,0,3,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,ATENDIMENTO FISIOTERAPÊUTICO EM PACIENTE C/ TR...
4,43,2022,1,3508528,4321109205897,0303040297,20211129,20211228,0302040013,127.00,430460,430460,0,0,20,CANOAS,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,ATENDIMENTO FISIOTERAPÊUTICO EM PACIENTE C/ TR...


# Arquivo "Geopandas"

In [47]:
dados = pd.read_csv('dados_limpos.csv')

In [48]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462630 entries, 0 to 462629
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            462630 non-null  int64  
 1   SP_AA            462630 non-null  int64  
 2   SP_MM            462630 non-null  int64  
 3   SP_CNES          462630 non-null  int64  
 4   SP_NAIH          462630 non-null  int64  
 5   SP_PROCREA       462630 non-null  int64  
 6   SP_DTINTER       462630 non-null  int64  
 7   SP_DTSAIDA       462630 non-null  int64  
 8   SP_ATOPROF       462630 non-null  int64  
 9   SP_VALATO        462630 non-null  float64
 10  SP_M_HOSP        462630 non-null  int64  
 11  SP_M_PAC         462630 non-null  int64  
 12  SP_DES_HOS       462630 non-null  int64  
 13  SP_DES_PAC       462630 non-null  int64  
 14  SP_QT_PROC       462630 non-null  int64  
 15  SP_CIDADE_H      462630 non-null  object 
 16  SP_CIDADE_P      462472 non-null  obje

In [49]:
nulos = dados.isnull().sum()
print(nulos)

SP_UF                0
SP_AA                0
SP_MM                0
SP_CNES              0
SP_NAIH              0
SP_PROCREA           0
SP_DTINTER           0
SP_DTSAIDA           0
SP_ATOPROF           0
SP_VALATO            0
SP_M_HOSP            0
SP_M_PAC             0
SP_DES_HOS           0
SP_DES_PAC           0
SP_QT_PROC           0
SP_CIDADE_H          0
SP_CIDADE_P        158
SP_CIDADE_P_EXT      0
SP_N_PROC            0
SP_N_ATOS            0
dtype: int64


In [50]:
municipios = pd.read_csv('municipios.csv', sep=',', encoding='utf-8')

In [51]:
dados = dados.dropna(subset=['SP_CIDADE_P'])

In [52]:
municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   5570 non-null   int64  
 1   nome          5570 non-null   object 
 2   latitude      5570 non-null   float64
 3   longitude     5570 non-null   float64
 4   ddd           5570 non-null   int64  
 5   fuso_horario  5570 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 261.2+ KB


In [53]:
municipios.head()

,codigo_ibge,nome,latitude,longitude,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,34,America/Sao_Paulo
2,5200100,Abadiânia,-16197.00000,-48.7057,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,91,America/Sao_Paulo


In [54]:
# Função para normalizar nomes de cidades
def normalize_city_name(name):
    if isinstance(name, str):
        return ''.join(c for c in unicodedata.normalize('NFKD', name) if not unicodedata.combining(c)).lower().strip()
    else:
        return ''

In [55]:
# Aplicar normalização às colunas relevantes
dados['SP_CIDADE_H_norm'] = dados['SP_CIDADE_H'].apply(normalize_city_name)
dados['SP_CIDADE_P_norm'] = dados['SP_CIDADE_P'].apply(normalize_city_name)
dados['SP_CIDADE_P_EXT_norm'] = dados['SP_CIDADE_P_EXT'].apply(normalize_city_name)
municipios['nome_norm'] = municipios['nome'].apply(normalize_city_name)

In [56]:
# Mesclar os DataFrames utilizando as colunas normalizadas
dados = dados.merge(municipios[['nome_norm', 'latitude', 'longitude']], how='left', left_on='SP_CIDADE_H_norm', right_on='nome_norm')
dados.rename(columns={'latitude': 'latitude_h', 'longitude': 'longitude_h'}, inplace=True)
dados.drop(columns=['SP_CIDADE_H_norm', 'nome_norm'], inplace=True)

dados = dados.merge(municipios[['nome_norm', 'latitude', 'longitude']], how='left', left_on='SP_CIDADE_P_norm', right_on='nome_norm')
dados.rename(columns={'latitude': 'latitude_p', 'longitude': 'longitude_p'}, inplace=True)
dados.drop(columns=['SP_CIDADE_P_norm', 'nome_norm'], inplace=True)

dados = dados.merge(municipios[['nome_norm', 'latitude', 'longitude']], how='left', left_on='SP_CIDADE_P_EXT_norm', right_on='nome_norm')
dados.rename(columns={'latitude': 'latitude_p_ext', 'longitude': 'longitude_p_ext'}, inplace=True)
dados.drop(columns=['SP_CIDADE_P_EXT_norm', 'nome_norm'], inplace=True)

In [57]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496473 entries, 0 to 496472
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            496473 non-null  int64  
 1   SP_AA            496473 non-null  int64  
 2   SP_MM            496473 non-null  int64  
 3   SP_CNES          496473 non-null  int64  
 4   SP_NAIH          496473 non-null  int64  
 5   SP_PROCREA       496473 non-null  int64  
 6   SP_DTINTER       496473 non-null  int64  
 7   SP_DTSAIDA       496473 non-null  int64  
 8   SP_ATOPROF       496473 non-null  int64  
 9   SP_VALATO        496473 non-null  float64
 10  SP_M_HOSP        496473 non-null  int64  
 11  SP_M_PAC         496473 non-null  int64  
 12  SP_DES_HOS       496473 non-null  int64  
 13  SP_DES_PAC       496473 non-null  int64  
 14  SP_QT_PROC       496473 non-null  int64  
 15  SP_CIDADE_H      496473 non-null  object 
 16  SP_CIDADE_P      496473 non-null  obje

In [58]:
nulos = dados.isnull().sum()

In [59]:
print(nulos)

SP_UF                   0
SP_AA                   0
SP_MM                   0
SP_CNES                 0
SP_NAIH                 0
SP_PROCREA              0
SP_DTINTER              0
SP_DTSAIDA              0
SP_ATOPROF              0
SP_VALATO               0
SP_M_HOSP               0
SP_M_PAC                0
SP_DES_HOS              0
SP_DES_PAC              0
SP_QT_PROC              0
SP_CIDADE_H             0
SP_CIDADE_P             0
SP_CIDADE_P_EXT         0
SP_N_PROC               0
SP_N_ATOS               0
latitude_h              0
longitude_h             0
latitude_p           1344
longitude_p          1344
latitude_p_ext     496473
longitude_p_ext    496473
dtype: int64


In [60]:
print(dados.columns)

Index(['SP_UF', 'SP_AA', 'SP_MM', 'SP_CNES', 'SP_NAIH', 'SP_PROCREA',
       'SP_DTINTER', 'SP_DTSAIDA', 'SP_ATOPROF', 'SP_VALATO', 'SP_M_HOSP',
       'SP_M_PAC', 'SP_DES_HOS', 'SP_DES_PAC', 'SP_QT_PROC', 'SP_CIDADE_H',
       'SP_CIDADE_P', 'SP_CIDADE_P_EXT', 'SP_N_PROC', 'SP_N_ATOS',
       'latitude_h', 'longitude_h', 'latitude_p', 'longitude_p',
       'latitude_p_ext', 'longitude_p_ext'],
      dtype='object')


In [61]:
dados = dados.loc[:,~dados.columns.duplicated()]

In [62]:
# Criando uma máscara booleana para identificar linhas onde a coluna SP_CIDADE_P é nula
mask = dados['latitude_p'].isnull()

# Aplicando a máscara para filtrar as linhas e selecionando as colunas desejadas para visualização
linhas_nulas = dados.loc[mask, ['SP_DES_HOS','SP_CIDADE_H', 'SP_CIDADE_P',
                             'SP_CIDADE_P_EXT', 'SP_N_PROC', 'SP_N_ATOS',
                             'latitude_h', 'longitude_h', 'latitude_p', 'longitude_p',
                             'latitude_p_ext', 'longitude_p_ext']]

# Imprimindo algumas linhas para visualização
print(linhas_nulas.head())

       SP_DES_HOS SP_CIDADE_H           SP_CIDADE_P SP_CIDADE_P_EXT  \
4403            1      CANOAS  Cidade de fora do RS    Cidade do RS   
4608            1      CANOAS  Cidade de fora do RS    Cidade do RS   
12715           1      CANOAS  Cidade de fora do RS    Cidade do RS   
12716           1      CANOAS  Cidade de fora do RS    Cidade do RS   
12717           1      CANOAS  Cidade de fora do RS    Cidade do RS   

                                               SP_N_PROC  \
4403   DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...   
4608              PARTO NORMAL EM GESTACAO DE ALTO RISCO   
12715  DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...   
12716  DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...   
12717  DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...   

                                               SP_N_ATOS  latitude_h  \
4403   DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...    -29.9128   
4608              PARTO NORMAL EM GESTACAO DE ALTO RISCO    -29.9128

In [63]:
# Remover duplicatas mantendo apenas a primeira ocorrência
dados = dados.drop_duplicates()

# Exibir o resultado
print(dados.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496473 entries, 0 to 496472
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            496473 non-null  int64  
 1   SP_AA            496473 non-null  int64  
 2   SP_MM            496473 non-null  int64  
 3   SP_CNES          496473 non-null  int64  
 4   SP_NAIH          496473 non-null  int64  
 5   SP_PROCREA       496473 non-null  int64  
 6   SP_DTINTER       496473 non-null  int64  
 7   SP_DTSAIDA       496473 non-null  int64  
 8   SP_ATOPROF       496473 non-null  int64  
 9   SP_VALATO        496473 non-null  float64
 10  SP_M_HOSP        496473 non-null  int64  
 11  SP_M_PAC         496473 non-null  int64  
 12  SP_DES_HOS       496473 non-null  int64  
 13  SP_DES_PAC       496473 non-null  int64  
 14  SP_QT_PROC       496473 non-null  int64  
 15  SP_CIDADE_H      496473 non-null  object 
 16  SP_CIDADE_P      496473 non-null  obje

In [64]:
# Substituir NaN em latitude_p e longitude_p por "cidades de fora RS"
dados['latitude_p_ext'].fillna('cidade do RS', inplace=True)
dados['longitude_p_ext'].fillna('cidade do RS', inplace=True)

# Exibir o resultado
print(dados.head())

   SP_UF  SP_AA  SP_MM  SP_CNES        SP_NAIH  SP_PROCREA  SP_DTINTER  \
0     43   2022      1  3508528  4321109205787   303010037    20211110   
1     43   2022      1  3508528  4321109205787   303010037    20211110   
2     43   2022      1  3508528  4321109205897   303040297    20211129   
3     43   2022      1  3508528  4321109205897   303040297    20211129   
4     43   2022      1  3508528  4321109205897   303040297    20211129   

   SP_DTSAIDA  SP_ATOPROF  SP_VALATO  ...  SP_CIDADE_P  SP_CIDADE_P_EXT  \
0    20211209   204060095       0.00  ...       CANOAS     Cidade do RS   
1    20211209   211020036       0.00  ...       CANOAS     Cidade do RS   
2    20211228   303040297      87.85  ...       CANOAS     Cidade do RS   
3    20211228   302040013      19.05  ...       CANOAS     Cidade do RS   
4    20211228   302040013     127.00  ...       CANOAS     Cidade do RS   

                                           SP_N_PROC  \
0           TRATAMENTO DE OUTRAS DOENÇAS BACTERI

/tmp/ipykernel_33377/4003231006.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['latitude_p_ext'].fillna('cidade do RS', inplace=True)
/tmp/ipykernel_33377/4003231006.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'cidade do RS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dados['latitude_p_ext'].fillna('cidade do RS', inplace=True)
/tmp/ipykernel_33377/4003231006.py:3: FutureWarning: A

In [65]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496473 entries, 0 to 496472
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            496473 non-null  int64  
 1   SP_AA            496473 non-null  int64  
 2   SP_MM            496473 non-null  int64  
 3   SP_CNES          496473 non-null  int64  
 4   SP_NAIH          496473 non-null  int64  
 5   SP_PROCREA       496473 non-null  int64  
 6   SP_DTINTER       496473 non-null  int64  
 7   SP_DTSAIDA       496473 non-null  int64  
 8   SP_ATOPROF       496473 non-null  int64  
 9   SP_VALATO        496473 non-null  float64
 10  SP_M_HOSP        496473 non-null  int64  
 11  SP_M_PAC         496473 non-null  int64  
 12  SP_DES_HOS       496473 non-null  int64  
 13  SP_DES_PAC       496473 non-null  int64  
 14  SP_QT_PROC       496473 non-null  int64  
 15  SP_CIDADE_H      496473 non-null  object 
 16  SP_CIDADE_P      496473 non-null  obje

In [66]:
contador = dados['longitude_p_ext'].value_counts()
print(contador)

longitude_p_ext
cidade do RS    496473
Name: count, dtype: int64


In [67]:
contagem_naih = dados['SP_NAIH'].nunique()

print(f"Quantidade de valores únicos na coluna SP_NAIH: {contagem_naih}")

Quantidade de valores únicos na coluna SP_NAIH: 31337


In [68]:
dados_sem_duplicatas = dados.drop_duplicates(subset=['SP_NAIH', 'SP_N_PROC'])

# Contar quantos procedimentos únicos cada paciente teve
contagem_procedimentos = dados_sem_duplicatas.groupby('SP_N_PROC')['SP_N_PROC'].size()

print(contagem_procedimentos)
valor = contagem_procedimentos.value_counts()
print(valor)

SP_N_PROC
AJUSTE MEDICAMENTOSO DE SITUACOES NEUROLOGICAS AGUDIZADAS                    27
ALARGAMENTO DA ENTRADA VAGINAL                                                1
ALONGAMENTO / ENCURTAMENTO MIOTENDINOSO                                       6
ALONGAMENTO E/OU TRANSPORTE OSSEO DE OSSOS LONGOS (EXCETO DA MAO E DO PE)    10
AMPUTACAO / DESARTICULACAO DE DEDO                                           94
                                                                             ..
VIDEOTORACOSCOPIA                                                             5
VITRECTOMIA ANTERIOR                                                         11
VULVECTOMIA AMPLIADA C/ LINFADENECTOMIA EM ONCOLOGIA                          1
VULVECTOMIA SIMPLES                                                           2
VULVECTOMIA SIMPLES POR TUMOR                                                 4
Name: SP_N_PROC, Length: 662, dtype: int64
SP_N_PROC
1       154
2        75
3        47
4        39
5        

In [69]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# 1. Criar as geometrias a partir dos dados originais
geometry = [Point(xy) for xy in zip(dados['longitude_h'], dados['latitude_h'])]

# 2. Criar o GeoDataFrame com o sistema de coordenadas WGS84 (EPSG:4326)
gdf = gpd.GeoDataFrame(dados, geometry=geometry, crs="EPSG:4326")

# 3. Filtrar valores inválidos (fora dos limites geográficos do sistema WGS84)
gdf = gdf[(gdf['longitude_h'] >= -180) & (gdf['longitude_h'] <= 180) &
          (gdf['latitude_h'] >= -90) & (gdf['latitude_h'] <= 90)]

# 4. Plotar os dados no mapa
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)
ax.set_extent([-74, -34, -35, 6], crs=ccrs.PlateCarree())  # Ajustar os limites para o Brasil
gdf.plot(ax=ax, marker='o', color='red', markersize=5, label='Internações')
plt.title('Mapa de Internações no Brasil')
plt.legend()
plt.show()


/tmp/ipykernel_33377/4234419596.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [70]:
# gdf = gdf.to_crs(epsg=4326)


In [71]:
print(gdf.total_bounds)
print(gdf.is_empty.any())  # Deve retornar False
print(gdf.is_valid.all())  # Deve retornar True
print(gdf.geometry.y.min(), gdf.geometry.y.max())
print(dados[['longitude_h', 'latitude_h']].head())
print(dados[['longitude_h', 'latitude_h']].describe())


[-51.1857 -29.9128 -51.1857 -29.9128]
False
True
-29.9128 -29.9128
   longitude_h  latitude_h
0     -51.1857    -29.9128
1     -51.1857    -29.9128
2     -51.1857    -29.9128
3     -51.1857    -29.9128
4     -51.1857    -29.9128
       longitude_h    latitude_h
count  496473.0000  4.964730e+05
mean      -51.1857 -2.991280e+01
std         0.0000  1.065815e-14
min       -51.1857 -2.991280e+01
25%       -51.1857 -2.991280e+01
50%       -51.1857 -2.991280e+01
75%       -51.1857 -2.991280e+01
max       -51.1857 -2.991280e+01


In [72]:
# 1. Criar as geometrias a partir dos dados originais
geometry = [Point(xy) for xy in zip(dados['longitude_h'], dados['latitude_h'])]

# 2. Criar o GeoDataFrame com o sistema de coordenadas WGS84 (EPSG:4326)
gdf = gpd.GeoDataFrame(dados, geometry=geometry, crs="EPSG:4326")

# 3. Filtrar valores inválidos (fora dos limites geográficos do sistema WGS84)
gdf = gdf[(gdf['longitude_h'] >= -180) & (gdf['longitude_h'] <= 180) &
          (gdf['latitude_h'] >= -90) & (gdf['latitude_h'] <= 90)]

# 4. Configurar o gráfico para usar o sistema de coordenadas
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Adicionar o mapa básico com limites de países
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)

# Limitar o gráfico ao Rio Grande do Sul (coordenadas aproximadas)
ax.set_extent([-57.65, -49.5, -34.0, -27.0], crs=ccrs.PlateCarree())  # Ajuste os limites para o RS

# Plotar os pontos do GeoDataFrame no mapa
gdf.plot(ax=ax, marker='o', color='red', markersize=5, label='Internações')

# Adicionar título e legendas
plt.title('Mapa de Internações no Rio Grande do Sul')
plt.legend()
plt.show()

/tmp/ipykernel_33377/3248777970.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [73]:
cidades_acima_menos25 = dados[dados['latitude_h'] < -25]

contagem_cidades_acima_menos25 = cidades_acima_menos25['SP_CIDADE_H'].nunique()

# Mostrar as linhas onde latitude_h está entre -25 e 0
linhas_latitude_entre_menos25_e_zero = dados[(dados['latitude_h'] > -25) & (dados['latitude_h'] <= 0)]

# Exibir os resultados
print(f"Cidades com latitude acima de -25: {contagem_cidades_acima_menos25}")
print("\nLinhas com latitude entre -25 e 0:")
print(linhas_latitude_entre_menos25_e_zero)

Cidades com latitude acima de -25: 1

Linhas com latitude entre -25 e 0:
Empty DataFrame
Columns: [SP_UF, SP_AA, SP_MM, SP_CNES, SP_NAIH, SP_PROCREA, SP_DTINTER, SP_DTSAIDA, SP_ATOPROF, SP_VALATO, SP_M_HOSP, SP_M_PAC, SP_DES_HOS, SP_DES_PAC, SP_QT_PROC, SP_CIDADE_H, SP_CIDADE_P, SP_CIDADE_P_EXT, SP_N_PROC, SP_N_ATOS, latitude_h, longitude_h, latitude_p, longitude_p, latitude_p_ext, longitude_p_ext]
Index: []

[0 rows x 26 columns]


In [74]:
# Filtrar as cidades com latitude acima de -25
cidades_acima_menos25 = dados[dados['latitude_h'] > -25]

# Selecionar apenas as colunas de interesse: nome da cidade e latitude
resultado = cidades_acima_menos25[['SP_CIDADE_H', 'latitude_h']]

# Exibir os resultados
print("Cidades com latitude acima de -25:")
print(resultado)

Cidades com latitude acima de -25:
Empty DataFrame
Columns: [SP_CIDADE_H, latitude_h]
Index: []


In [75]:
# Filtrar as cidades com latitude acima de -25
cidades_acima_menos25 = dados[dados['latitude_h'] > -25]

# Contar o número de cidades únicas
numero_cidades_acima_menos25 = cidades_acima_menos25['SP_CIDADE_H'].nunique()

# Exibir o resultado
print(f"Número de cidades com latitude acima de -25: {numero_cidades_acima_menos25}")

Número de cidades com latitude acima de -25: 0


In [76]:
# Filtrar as cidades com latitude acima de -25
cidades_acima_menos25 = dados[dados['latitude_h'] > -25]

# Obter as informações relevantes (nome, latitude, longitude)
cidades_info = cidades_acima_menos25[['SP_CIDADE_H', 'latitude_h', 'longitude_h']].drop_duplicates()

# Exibir o resultado
print("Cidades com latitude acima de -25:")
for index, row in cidades_info.iterrows():
    print(f"Nome: {row['SP_CIDADE_H']}, Latitude: {row['latitude_h']}, Longitude: {row['longitude_h']}")

Cidades com latitude acima de -25:


In [77]:
# Verificar se existe a cidade "Lajeado do Bugre"
cidade_lajeado_bugre = dados[dados['SP_CIDADE_P'] == 'MARAU']

# Exibir o resultado
if not cidade_lajeado_bugre.empty:
    print("A cidade 'Lajeado do Bugre' foi encontrada.")
else:
    print("A cidade 'Lajeado do Bugre' não foi encontrada.")

A cidade 'Lajeado do Bugre' foi encontrada.


In [78]:
import pandas as pd

# Carregar o arquivo municipios.csv
municipios = pd.read_csv('municipios.csv')

# Filtrar as cidades fora do Rio Grande do Sul com latitude > -25
cidades_foras_rgs = dados[dados['latitude_h'] > -25]

# Criar uma lista para armazenar os resultados
resultados = []

# Iterar sobre as cidades fora do Rio Grande do Sul
for index, row in cidades_foras_rgs.iterrows():
    nome_cidade = row['SP_CIDADE_H']
    latitude_for = row['latitude_h']
    
    # Verificar se existe uma cidade com o mesmo nome e latitude < -25 no municipios.csv
    cidade_info = municipios[municipios['nome'] == nome_cidade]
    cidade_rgs = cidade_info[cidade_info['latitude'] < -25]
    
    # Se a cidade for encontrada, adicionar à lista de resultados
    if not cidade_rgs.empty:
        resultados.append((nome_cidade, latitude_for))

# Exibir os resultados
if resultados:
    print("Cidades fora do Rio Grande do Sul com latitude maior que -25 que têm cidades com o mesmo nome no RS:")
    for cidade, lat in resultados:
        print(f"{cidade} (latitude fora do RS: {lat})")
else:
    print("Não foram encontradas cidades fora do Rio Grande do Sul com latitude maior que -25 que têm cidades com o mesmo nome no RS.")

Não foram encontradas cidades fora do Rio Grande do Sul com latitude maior que -25 que têm cidades com o mesmo nome no RS.


In [79]:
import pandas as pd
cidades_procuradas = [
    "ALVORADA", "CACHOEIRINHA", "CACHOEIRINHA", "LAJEADO", "VERA CRUZ",
    "VERA CRUZ", "VERA CRUZ", "TRIUNFO", "TRIUNFO", "SÃO FRANCISCO DE PAULA",
    "BOM JESUS", "BOM JESUS", "BOM JESUS", "SOBRADINHO", "PLANALTO",
    "PLANALTO", "CRUZEIRO DO SUL", "CRUZEIRO DO SUL", "CAIÇARA", "SANTA MARIA",
    "MARAU", "SÃO GABRIEL", "HUMAITÁ", "TAPEJARA", "SARANDI", "SOLEDADE",
    "SARANDI", "LAJEADO", "PLANALTO", "MARAU", "SOLEDADE",
    "CACHOEIRINHA", "ALVORADA", "SÃO GABRIEL", "SOBRADINHO",
    "SANTA MARIA", "SÃO FRANCISCO DE PAULA", "VERA CRUZ",
    "TAPEJARA", "TRIUNFO", "HUMAITÁ", "CRUZEIRO DO SUL",
    "BOM JESUS", "CAIÇARA"
]

# Criar uma lista para armazenar as informações encontradas
cidades_info = []

# Iterar sobre as cidades procuradas
for cidade in cidades_procuradas:
    # Filtrar as linhas correspondentes no DataFrame de municípios (sem considerar maiúsculas/minúsculas)
    cidade_info = municipios[municipios['nome'].str.lower() == cidade.lower()]

    # Verificar se a cidade foi encontrada e se sua latitude é menor que -25
    if not cidade_info.empty:
        lat = cidade_info['latitude'].values[0]
        lon = cidade_info['longitude'].values[0]
        
        if lat < -25:  # Verifica se a latitude é menor que -25
            cidades_info.append((cidade, lat, lon))

# Exibir os resultados
print("Cidades com latitude menor que -25:")
for cidade, lat, lon in cidades_info:
    print(f"Nome: {cidade}, Latitude: {lat}, Longitude: {lon}")

Cidades com latitude menor que -25:
Nome: TRIUNFO, Latitude: -29.9291, Longitude: -51.7075
Nome: TRIUNFO, Latitude: -29.9291, Longitude: -51.7075
Nome: SÃO FRANCISCO DE PAULA, Latitude: -29.4404, Longitude: -50.5828
Nome: BOM JESUS, Latitude: -26.7326, Longitude: -52.3919
Nome: BOM JESUS, Latitude: -26.7326, Longitude: -52.3919
Nome: BOM JESUS, Latitude: -26.7326, Longitude: -52.3919
Nome: CAIÇARA, Latitude: -27.2791, Longitude: -53.4257
Nome: SANTA MARIA, Latitude: -29.6868, Longitude: -53.8149
Nome: MARAU, Latitude: -28.4498, Longitude: -52.1986
Nome: SÃO GABRIEL, Latitude: -30.3337, Longitude: -54.3217
Nome: HUMAITÁ, Latitude: -27.5691, Longitude: -53.9695
Nome: MARAU, Latitude: -28.4498, Longitude: -52.1986
Nome: SÃO GABRIEL, Latitude: -30.3337, Longitude: -54.3217
Nome: SANTA MARIA, Latitude: -29.6868, Longitude: -53.8149
Nome: SÃO FRANCISCO DE PAULA, Latitude: -29.4404, Longitude: -50.5828
Nome: TRIUNFO, Latitude: -29.9291, Longitude: -51.7075
Nome: HUMAITÁ, Latitude: -27.5691, L

In [80]:
import pandas as pd

# Supondo que o DataFrame 'dados' já está carregado

# Lista consolidada de cidades com suas respectivas coordenadas
cidades_coordenadas = [
    {"cidade": "INDEPENDÊNCIA", "latitude": -27.8354, "longitude": -54.1886},
    {"cidade": "COLORADO", "latitude": -28.5258, "longitude": -52.9928},
    {"cidade": "NOVA SANTA RITA", "latitude": -29.8525, "longitude": -51.2837},
    {"cidade": "JACUTINGA", "latitude": -27.7291, "longitude": -52.5372},
    {"cidade": "CENTENÁRIO", "latitude": -27.7615, "longitude": -51.9984},
    {"cidade": "SOLEDADE", "latitude": -28.8306, "longitude": -52.5131},
    {"cidade": "SÃO MARTINHO", "latitude": -27.7112, "longitude": -53.9699},
    {"cidade": "SÃO GABRIEL", "latitude": -30.3337, "longitude": -54.3217},
    {"cidade": "HUMAITÁ", "latitude": -27.5691, "longitude": -53.9695},
    {"cidade": "BOM JESUS", "latitude": -28.6697, "longitude": -50.4295},
    {"cidade": "SANTA MARIA", "latitude": -29.6868, "longitude": -53.8149},
    {"cidade": "SÃO FRANCISCO DE PAULA", "latitude": -29.4404, "longitude": -50.5828},
    {"cidade": "TRIUNFO", "latitude": -29.9291, "longitude": -51.7075},
    {"cidade": "CAIÇARA", "latitude": -27.2791, "longitude": -53.4257},
    {"cidade": "MARAU", "latitude": -28.4498, "longitude": -52.1986},
    {"cidade": "ALTO ALEGRE", "latitude": -28.7769, "longitude": -52.9893},
    {"cidade": "COLINAS", "latitude": -29.3948, "longitude": -51.8556},
    {"cidade": "TAVARES", "latitude": -31.2843, "longitude": -51.0880},
]

# Loop para atualizar as latitudes e longitudes no DataFrame
for cidade_info in cidades_coordenadas:
    cidade = cidade_info["cidade"].lower()
    nova_latitude = cidade_info["latitude"]
    nova_longitude = cidade_info["longitude"]

    # Atualizar latitude e longitude para 'SP_CIDADE_H'
    dados.loc[dados['SP_CIDADE_H'].str.lower() == cidade, ['latitude_h', 'longitude_h']] = [nova_latitude, nova_longitude]

    # Atualizar latitude e longitude para 'SP_CIDADE_P'
    dados.loc[dados['SP_CIDADE_P'].str.lower() == cidade, ['latitude_p', 'longitude_p']] = [nova_latitude, nova_longitude]


In [81]:
import pandas as pd

# Supondo que o DataFrame 'dados' já está carregado

# Criar um conjunto para armazenar as informações únicas
cidades_info = set()

# Verificar todas as colunas de latitude e suas respectivas longitudes e cidades
for index, row in dados.iterrows():
 
    # Verificar 'SP_CIDADE_P', 'latitude_p', 'longitude_p'
    if row['latitude_p'] > -25:
        cidades_info.add((row['SP_CIDADE_P'], row['latitude_p'], row['longitude_p']))



# Exibir os resultados
print("Cidades com latitude maior que -25 (únicas):")
for cidade, lat, lon in cidades_info:
    print(f"Nome: {cidade}, Latitude: {lat}, Longitude: {lon}")

Cidades com latitude maior que -25 (únicas):
Nome: PLANALTO, Latitude: -14.6654, Longitude: -40.4718
Nome: CACHOEIRINHA, Latitude: -6.1156, Longitude: -47.9234
Nome: VERA CRUZ, Latitude: -6.04399, Longitude: -35428.0
Nome: VERA CRUZ, Latitude: -22.2183, Longitude: -49.8207
Nome: PLANALTO, Latitude: -21.0342, Longitude: -49933.0
Nome: CACHOEIRINHA, Latitude: -8.48668, Longitude: -36.2402
Nome: VERA CRUZ, Latitude: -12.9568, Longitude: -38.6153
Nome: ALVORADA, Latitude: -12.4785, Longitude: -49.1249
Nome: CRUZEIRO DO SUL, Latitude: -7.62762, Longitude: -72.6756
Nome: CRUZEIRO DO SUL, Latitude: -22.9624, Longitude: -52.1622
Nome: LAJEADO, Latitude: -9.74996, Longitude: -48.3565
Nome: SOBRADINHO, Latitude: -9.45024, Longitude: -40.8145
Nome: TAPEJARA, Latitude: -23.7315, Longitude: -52.8735


In [82]:
import pandas as pd

# Supondo que o DataFrame 'dados' já está carregado

# Criar uma lista para armazenar as cidades com longitude maior que -49
cidades_longitude_maior_49 = []

# Iterar sobre as linhas do DataFrame para encontrar cidades com longitude maior que -49
for index, row in dados.iterrows():
    if row['longitude_h'] > -49:  # Verifica se a longitude é maior que -49
        cidades_longitude_maior_49.append(row['SP_CIDADE_H'])

# Obter apenas os nomes únicos das cidades
cidades_longitude_maior_49_unicas = list(set(cidades_longitude_maior_49))

# Exibir os resultados
print("Cidades com longitude maior que -49:")
for cidade in cidades_longitude_maior_49_unicas:
    print(cidade)

Cidades com longitude maior que -49:


In [83]:
# Definir os limites do Rio Grande do Sul
latitude_min = -33.0
latitude_max = -27.0
longitude_min = -57.0
longitude_max = -49.0

# Filtrar cidades fora dos limites do Rio Grande do Sul
cidades_fora = dados[
    ~((dados['latitude_h'].between(latitude_min, latitude_max)) &
      (dados['longitude_h'].between(longitude_min, longitude_max)))
]

print(cidades_fora[['SP_CIDADE_H', 'latitude_h', 'longitude_h']].drop_duplicates())

Empty DataFrame
Columns: [SP_CIDADE_H, latitude_h, longitude_h]
Index: []


In [84]:
inconsistencias = dados[
    (dados['latitude_h'] > 0) |  # Latitudes positivas não estão no Brasil
    (dados['longitude_h'] > 0) |  # Longitudes positivas estão fora do Brasil
    (dados['latitude_h'] < -35) |  # Latitudes muito ao sul
    (dados['latitude_h'] > -25) |  # Latitudes muito ao norte
    (dados['longitude_h'] < -60) |  # Longitudes muito ao oeste
    (dados['longitude_h'] > -45)  # Longitudes muito ao leste
]
# coordenadas problemáticas
print(inconsistencias[['SP_CIDADE_H', 'latitude_h', 'longitude_h']].drop_duplicates())


Empty DataFrame
Columns: [SP_CIDADE_H, latitude_h, longitude_h]
Index: []


In [85]:
# Dicionário com as coordenadas corretas para as cidades
correcao_coordenadas = {
    "GARIBALDI": {"latitude_h": -29.259, "longitude_h": -51.5352},
    "ALVORADA": {"latitude_h": -29.9914, "longitude_h": -51.0809},
    "CACHOEIRA DO SUL": {"latitude_h": -30.0331, "longitude_h": -52.8926},
    "SÃO LUIZ GONZAGA": {"latitude_h": -28.4088, "longitude_h": -54.9559},
    "RIO PARDO": {"latitude_h": -29.9881, "longitude_h": -52.3712},
    "SÃO MIGUEL DAS MISSÕES": {"latitude_h": -28.5561, "longitude_h": -54.5559},
    "SANTO CRISTO": {"latitude_h": -27.8263, "longitude_h": -54.662},
    "RIOZINHO": {"latitude_h": -29.639, "longitude_h": -50.4488},
    "SANTA CRUZ DO SUL": {"latitude_h": -29.722, "longitude_h": -52.4343},
    "MUÇUM": {"latitude_h": -29.1633, "longitude_h": -51.8715},
    "SAPUCAIA DO SUL": {"latitude_h": -29.8334, "longitude_h": -51.1441},
    "DOIS LAJEADOS": {"latitude_h": -28.983, "longitude_h": -51.8396},
    "ENCRUZILHADA DO SUL": {"latitude_h": -30.5433, "longitude_h": -52.5204},
    "CRUZ ALTA": {"latitude_h": -28.645, "longitude_h": -53.6048},
    "CRUZEIRO DO SUL": {"latitude_h": -29.516, "longitude_h": -52.0788},
    "CANGUÇU": {"latitude_h": -31.396, "longitude_h": -52.6783},
    "SANANDUVA": {"latitude_h": -27.9473, "longitude_h": -51.8078},
    "IJUÍ": {"latitude_h": -28.3881, "longitude_h": -53.9201},
    "SANTA BÁRBARA DO SUL": {"latitude_h": -28.3654, "longitude_h": -53.2511},
    "JAGUARÃO": {"latitude_h": -32.5604, "longitude_h": -53.3778},
    "SANTA VITÓRIA DO PALMAR": {"latitude_h": -33.525, "longitude_h": -53.3717},
    "JAGUARI": {"latitude_h": -29.4936, "longitude_h": -54.703},
    "HERVAL": {"latitude_h": -32.0242, "longitude_h": -53.3944},
    "CACEQUI": {"latitude_h": -29.8883, "longitude_h": -54.822},
    "TAPEJARA": {"latitude_h": -28.065, "longitude_h": -52.009},
    "NOVA PALMA": {"latitude_h": -29.471, "longitude_h": -53.4689},
    "CONSTANTINA": {"latitude_h": -27.7324, "longitude_h": -52.9938},
    "SARANDI": {"latitude_h": -27.942, "longitude_h": -52.9231},
    "PARAÍSO DO SUL": {"latitude_h": -29.6717, "longitude_h": -53.1444},
    "CANELA": {"latitude_h": -29.356, "longitude_h": -50.8119},
    "MAXIMILIANO DE ALMEIDA": {"latitude_h": -27.6325, "longitude_h": -51.8018},
    "PAIM FILHO": {"latitude_h": -27.7075, "longitude_h": -51.763},
    "PALMITINHO": {"latitude_h": -27.3596, "longitude_h": -53.558},
    "CHIAPETTA": {"latitude_h": -28.923, "longitude_h": -53.9419},
    "AMETISTA DO SUL": {"latitude_h": -27.3607, "longitude_h": -53.183},
    "SALDANHA MARINHO": {"latitude_h": -28.3941, "longitude_h": -53.097},
    "PINHAL GRANDE": {"latitude_h": -29.345, "longitude_h": -53.3206},
    "QUARAÍ": {"latitude_h": -30.3844, "longitude_h": -56.4483},
    "ESTEIO": {"latitude_h": -29.8522, "longitude_h": -51.184}
}

# Atualizar as coordenadas no DataFrame
for cidade, coords in correcao_coordenadas.items():
    # Atualizar para SP_CIDADE_H
    dados.loc[dados['SP_CIDADE_H'].str.upper() == cidade, ['latitude_h', 'longitude_h']] = [coords['latitude_h'], coords['longitude_h']]
    # Se houver uma coluna para SP_CIDADE_P, também pode ser atualizada da mesma maneira
    if 'latitude_p' in dados.columns and 'longitude_p' in dados.columns:
        dados.loc[dados['SP_CIDADE_P'].str.upper() == cidade, ['latitude_p', 'longitude_p']] = [coords['latitude_h'], coords['longitude_h']]


In [86]:
# Adicionar as coordenadas corretas para as cidades
correcao_coordenadas = {
    "CACHOEIRINHA": {"latitude": -29.9501, "longitude": -51.0913},
    "LAJEADO": {"latitude": -29.4632, "longitude": -51.9644},
    "SOBRADINHO": {"latitude": -29.4194, "longitude": -52.6267},
    "VERA CRUZ": {"latitude": -29.7183, "longitude": -52.5154},
    "PLANALTO": {"latitude": -27.3297, "longitude": -53.0575},
}

# Atualizar as coordenadas no DataFrame
for cidade, coords in correcao_coordenadas.items():
    # Atualizar para SP_CIDADE_H
    dados.loc[dados['SP_CIDADE_H'].str.upper() == cidade, ['latitude_h', 'longitude_h']] = [coords['latitude'], coords['longitude']]
    # Atualizar para SP_CIDADE_P (se necessário, ajuste a coluna)
    dados.loc[dados['SP_CIDADE_P'].str.upper() == cidade, ['latitude_p', 'longitude_p']] = [coords['latitude'], coords['longitude']]


In [87]:
# 1. Criar as geometrias a partir dos dados originais
geometry = [Point(xy) for xy in zip(dados['longitude_h'], dados['latitude_h'])]

# 2. Criar o GeoDataFrame com o sistema de coordenadas WGS84 (EPSG:4326)
gdf = gpd.GeoDataFrame(dados, geometry=geometry, crs="EPSG:4326")

# 3. Filtrar valores inválidos (fora dos limites geográficos do sistema WGS84)
gdf = gdf[(gdf['longitude_h'] >= -180) & (gdf['longitude_h'] <= 180) &
          (gdf['latitude_h'] >= -90) & (gdf['latitude_h'] <= 90)]

# 4. Configurar o gráfico para usar o sistema de coordenadas
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Adicionar o mapa básico com limites de países
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)

# Limitar o gráfico ao Rio Grande do Sul (coordenadas aproximadas)
ax.set_extent([-57.65, -49.5, -34.0, -27.0], crs=ccrs.PlateCarree())  # Ajuste os limites para o RS

# Plotar os pontos do GeoDataFrame no mapa
gdf.plot(ax=ax, marker='o', color='red', markersize=5, label='Internações')

# Adicionar título e legendas
plt.title('Mapa de Internações no Rio Grande do Sul', fontsize=16)
plt.legend(loc='upper left', fontsize=12)

# Exibir o gráfico
plt.show()

/tmp/ipykernel_33377/392953639.py:29: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [88]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# 1. Criar as geometrias a partir dos dados originais
geometry = [Point(xy) for xy in zip(dados['longitude_h'], dados['latitude_h'])]

# 2. Criar o GeoDataFrame com o sistema de coordenadas WGS84 (EPSG:4326)
gdf = gpd.GeoDataFrame(dados, geometry=geometry, crs="EPSG:4326")

# 3. Filtrar valores inválidos (fora dos limites geográficos do sistema WGS84)
gdf = gdf[(gdf['longitude_h'] >= -180) & (gdf['longitude_h'] <= 180) &
          (gdf['latitude_h'] >= -90) & (gdf['latitude_h'] <= 90)]

# 4. Plotar os dados no mapa
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)
ax.set_extent([-74, -34, -35, 6], crs=ccrs.PlateCarree())  # Ajustar os limites para o Brasil
gdf.plot(ax=ax, marker='o', color='red', markersize=5, label='Internações')
plt.title('Mapa de Internações no Brasil')
plt.legend()
plt.show()


/tmp/ipykernel_33377/4234419596.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [89]:
dados.to_csv('dados_RS.csv', index=False)
print("Arquivo 'dados_RS.csv' atualizado com sucesso!")

Arquivo 'dados_RS.csv' atualizado com sucesso!


# Verificações extras pós download dos dados

In [90]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496473 entries, 0 to 496472
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SP_UF            496473 non-null  int64  
 1   SP_AA            496473 non-null  int64  
 2   SP_MM            496473 non-null  int64  
 3   SP_CNES          496473 non-null  int64  
 4   SP_NAIH          496473 non-null  int64  
 5   SP_PROCREA       496473 non-null  int64  
 6   SP_DTINTER       496473 non-null  int64  
 7   SP_DTSAIDA       496473 non-null  int64  
 8   SP_ATOPROF       496473 non-null  int64  
 9   SP_VALATO        496473 non-null  float64
 10  SP_M_HOSP        496473 non-null  int64  
 11  SP_M_PAC         496473 non-null  int64  
 12  SP_DES_HOS       496473 non-null  int64  
 13  SP_DES_PAC       496473 non-null  int64  
 14  SP_QT_PROC       496473 non-null  int64  
 15  SP_CIDADE_H      496473 non-null  object 
 16  SP_CIDADE_P      496473 non-null  obje

In [91]:
dados.head()

,SP_UF,SP_AA,SP_MM,SP_CNES,SP_NAIH,SP_PROCREA,SP_DTINTER,SP_DTSAIDA,SP_ATOPROF,SP_VALATO,...,SP_CIDADE_P,SP_CIDADE_P_EXT,SP_N_PROC,SP_N_ATOS,latitude_h,longitude_h,latitude_p,longitude_p,latitude_p_ext,longitude_p_ext
0,43,2022,1,3508528,4321109205787,303010037,20211110,20211209,204060095,0.00,...,CANOAS,Cidade do RS,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,RADIOGRAFIA DE BACIA,-29.9128,-51.1857,-29.9128,-51.1857,cidade do RS,cidade do RS
1,43,2022,1,3508528,4321109205787,303010037,20211110,20211209,211020036,0.00,...,CANOAS,Cidade do RS,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,ELETROCARDIOGRAMA,-29.9128,-51.1857,-29.9128,-51.1857,cidade do RS,cidade do RS
2,43,2022,1,3508528,4321109205897,303040297,20211129,20211228,303040297,87.85,...,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,-29.9128,-51.1857,-29.9128,-51.1857,cidade do RS,cidade do RS
3,43,2022,1,3508528,4321109205897,303040297,20211129,20211228,302040013,19.05,...,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,ATENDIMENTO FISIOTERAPÊUTICO EM PACIENTE C/ TR...,-29.9128,-51.1857,-29.9128,-51.1857,cidade do RS,cidade do RS
4,43,2022,1,3508528,4321109205897,303040297,20211129,20211228,302040013,127.00,...,CANOAS,Cidade do RS,TRATAMENTO DE PROCESSO TOXI-INFECCIOSO DO CERE...,ATENDIMENTO FISIOTERAPÊUTICO EM PACIENTE C/ TR...,-29.9128,-51.1857,-29.9128,-51.1857,cidade do RS,cidade do RS


In [92]:
# Obter cidades únicas a partir da coluna SP_CIDADE_H
cidades_disponiveis = dados["SP_CIDADE_H"].unique()

# Mostrar a lista de cidades
print(f"Cidades disponíveis ({len(cidades_disponiveis)} no total):")
print(cidades_disponiveis)


# Mostrar a lista de cidades de pessoas que foram nos hospitais
cidades_disponiveis2 = dados["SP_CIDADE_P"].unique()
print(f"Cidades disponíveis ({len(cidades_disponiveis2)} no total):")
print(cidades_disponiveis2)


Cidades disponíveis (1 no total):
['CANOAS']
Cidades disponíveis (175 no total):
['CANOAS' 'SANTA ROSA' 'SAPUCAIA DO SUL' 'NOVA SANTA RITA' 'GRAVATAÍ'
 'MONTENEGRO' 'MARAU' 'IVOTI' 'TAQUARI' 'ENCANTADO' 'PRESIDENTE LUCENA'
 'ESTEIO' 'TAQUARA' 'IGREJINHA' 'NOVO HAMBURGO' 'PORTO ALEGRE'
 'ESTÂNCIA VELHA' 'PORTÃO' 'PAROBÉ' 'TEUTÔNIA' 'SÃO LEOPOLDO' 'ARARICÁ'
 'NOVA HARTZ' 'TRÊS COROAS' 'DOIS IRMÃOS' 'CAMPO BOM' 'TRIUNFO'
 'SÃO FRANCISCO DE PAULA' 'SANTA MARIA DO HERVAL' 'CAPELA DE SANTANA'
 'SAPIRANGA' 'SÃO SEBASTIÃO DO CAÍ' 'Cidade de fora do RS' 'RIOZINHO'
 'LINDOLFO COLLOR' 'CACHOEIRINHA' 'SANTO ANTÔNIO DA PATRULHA' 'CAPITÃO'
 'ESTRELA' 'ALVORADA' 'BROCHIER' 'ROLANTE' 'ELDORADO DO SUL' 'BAGÉ'
 'CIDREIRA' 'MORRO REUTER' 'CAMBARÁ DO SUL' 'PARECI NOVO'
 'SALVADOR DO SUL' 'ARROIO DO MEIO' 'LAJEADO' 'POÇO DAS ANTAS'
 'POUSO NOVO' 'CAÇAPAVA DO SUL' 'SÃO JOSÉ DO SUL' 'SANTA CLARA DO SUL'
 'BOM RETIRO DO SUL' 'VIAMÃO' 'BARÃO' 'VERA CRUZ' 'SÃO PEDRO DA SERRA'
 'BOQUEIRÃO DO LEÃO' 'TABAÍ' 'ARVOR

In [93]:
# Obter cidades únicas a partir da coluna SP_CIDADE_H
cidades_disponiveis = dados["SP_CIDADE_H"].unique()

# Mostrar a lista de cidades
#print(f"Cidades disponíveis ({len(cidades_disponiveis)} no total):")
#print(cidades_disponiveis)

# Criar o dicionário de mapeamento entre SP_CIDADE_H e SP_M_HOSP
mapeamento_cidades = {}
for cidade in cidades_disponiveis:
    # Filtrar os códigos correspondentes para cada cidade
    codigos = dados[dados["SP_CIDADE_H"] == cidade]["SP_M_HOSP"].unique()
    if len(codigos) > 0:
        mapeamento_cidades[cidade] = codigos[0]  # Considera o primeiro código encontrado
    else:
        mapeamento_cidades[cidade] = None  # Marca como None se não houver código

# Mostrar o dicionário de mapeamento
print("\nDicionário de mapeamento (cidade: código):")
for cidade, codigo in mapeamento_cidades.items():
    print(f"{cidade}: {codigo}")



Dicionário de mapeamento (cidade: código):
CANOAS: 430460


# PLOTS do DASH

In [94]:
# 1. Importações necessárias (adicione no início do notebook)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import calendar

# 2. Processamento inicial das datas
# Adicione após carregar os dados
dados['SP_DTINTER'] = pd.to_datetime(dados['SP_DTINTER'], format='%Y%m%d')
dados['SP_DTSAIDA'] = pd.to_datetime(dados['SP_DTSAIDA'], format='%Y%m%d')
dados['tempo_internacao'] = (dados['SP_DTSAIDA'] - dados['SP_DTINTER']).dt.days

In [95]:
# # 3. Análise temporal de internações
# plt.figure(figsize=(15, 8))
# internacoes_mes = dados['SP_DTINTER'].dt.month.value_counts().sort_index()
# internacoes_mes.index = [calendar.month_name[m] for m in internacoes_mes.index]
# sns.barplot(x=internacoes_mes.index, y=internacoes_mes.values)
# plt.title('Distribuição de Internações por Mês')
# plt.xticks(rotation=45)
# plt.ylabel('Número de Internações')
# plt.tight_layout()
# plt.savefig('internacoes_mes.png')
# plt.close()

In [96]:
# # 4. Top 10 procedimentos mais frequentes
# plt.figure(figsize=(15, 8))
# top_procedimentos = dados['SP_N_PROC'].value_counts().head(10)
# sns.barplot(x=top_procedimentos.values, y=top_procedimentos.index)
# plt.title('Top 10 Procedimentos Mais Frequentes')
# plt.xlabel('Número de Ocorrências')
# plt.tight_layout()
# plt.savefig('top_procedimentos.png')
# plt.close()

In [97]:
# # 5. Distribuição do tempo de internação
# plt.figure(figsize=(12, 6))
# sns.histplot(data=dados, x='tempo_internacao', bins=30)
# plt.title('Distribuição do Tempo de Internação')
# plt.xlabel('Dias de Internação')
# plt.ylabel('Frequência')
# plt.tight_layout()
# plt.savefig('tempo_internacao.png')
# plt.close()

In [98]:
# # 6. Análise de valores (com ajustes para melhor visualização)
# plt.figure(figsize=(12, 6))  # Aumentar o tamanho para melhor legibilidade
# sns.boxplot(data=dados, y='SP_VALATO')
# plt.title('Distribuição dos Valores dos Procedimentos')
# plt.ylabel('Valor (R$)')

# # Limitar o eixo Y para focar nos valores mais comuns e evitar outliers extremos
# plt.ylim(0, dados['SP_VALATO'].quantile(0.99))  # Mostra até o 99º percentil (em vez de 95%)
# plt.tight_layout()
# plt.savefig('valores_procedimentos.png')
# plt.close()

In [99]:
# import plotly.express as px
# import pandas as pd

# # 7. Gráfico interativo de valores por procedimento (usando Plotly)
# # Agrupar por procedimento e calcular a média e o total de valores
# valores_por_procedimento = dados.groupby('SP_N_PROC')['SP_VALATO'].agg(['mean', 'sum']).reset_index()
# valores_por_procedimento = valores_por_procedimento.sort_values(by='sum', ascending=False).head(10)  # Top 10 procedimentos por valor total

# # Criar uma coluna com nomes completos para tooltips
# valores_por_procedimento['Nome Completo'] = valores_por_procedimento['SP_N_PROC']


# # Gerar o gráfico de barras agrupadas, separando 'mean' e 'sum' em traces distintos
# fig = px.bar(
#     valores_por_procedimento, 
#     x='SP_N_PROC', 
#     y='mean',  # Usar apenas 'mean' inicialmente para teste
#     title='Distribuição Média e Total de Valores por Procedimento (Top 10)',
#     labels={'value': 'Valor (R$)', 'SP_N_PROC': 'Procedimento'},
#     hover_data={'SP_N_PROC': False, 'Nome Completo': True, 'mean': ':.2f', 'sum': ':.2f'}  # Mostrar nomes completos e valores no hover
# )

# # Adicionar barras para 'sum' separadamente para garantir visibilidade
# fig.add_bar(
#     x=valores_por_procedimento['SP_N_PROC'],
#     y=valores_por_procedimento['sum'],
#     name='Total dos Valores',
#     marker_color='#ff7f0e',  # Cor laranja para 'sum'
#     hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Métrica:</b> Total dos Valores<br><b>Valor (R$):</b> %{y:,.2f}<extra></extra>',
#     customdata=valores_por_procedimento['Nome Completo']  # Nomes completos para tooltips
# )

# # Ajustar os labels no eixo X para mostrar uma versão abreviada
# fig.update_xaxes(
#     tickangle=45, 
#     tickmode='array', 
#     tickvals=valores_por_procedimento['SP_N_PROC'], 
#     ticktext=[proc[:20] + '...' if len(proc) > 20 else proc for proc in valores_por_procedimento['SP_N_PROC']],  # Truncar labels longos
#     title_standoff=15
# )

# # Ajustar layout para clareza e legibilidade
# fig.update_layout(
#     height=600,  # Consistente com o app
#     width=800,  # Consistente com o app
#     xaxis_title=None,
#     yaxis_title="Valor (R$)",
#     showlegend=True,
#     legend_title_text='Métrica',  # Adicionar título à legenda
#     legend=dict(
#         x=1.05,  # Posicionar a legenda à direita
#         y=1.0,   # Alinhar ao topo
#         bgcolor='rgba(255, 255, 255, 0.5)',  # Fundo semi-transparente
#         bordercolor='Black',
#         borderwidth=1
#     ),
#     bargap=0.2,  # Espaçamento entre barras agrupadas
#     barmode='group'  # Garantir barras agrupadas
# )

# # Garantir que o gráfico seja exibido
# fig.show()
# fig.write_html('valores_por_procedimento.html', auto_open=False)

In [100]:
# # 8. Mapa de calor temporal
# # Criar matriz de contagem por hora do dia e dia da semana
# dados['hora'] = dados['SP_DTINTER'].dt.hour
# dados['dia_semana'] = dados['SP_DTINTER'].dt.day_name()

# heatmap_data = pd.crosstab(dados['dia_semana'], dados['hora'])
# plt.figure(figsize=(15, 8))
# sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt='d')
# plt.title('Distribuição de Internações por Hora e Dia da Semana')
# plt.tight_layout()
# plt.savefig('heatmap_temporal.png')
# plt.close()

In [101]:
# # 9. Análise de tendência temporal
# plt.figure(figsize=(15, 6))
# dados.set_index('SP_DTINTER')['SP_VALATO'].resample('W').mean().plot()
# plt.title('Tendência do Valor Médio dos Procedimentos ao Longo do Tempo')
# plt.xlabel('Data')
# plt.ylabel('Valor Médio (R$)')
# plt.tight_layout()
# plt.savefig('tendencia_temporal.png')
# plt.close()

In [102]:
# # 10. Dashboard interativo com Plotly
# f# Criar figura com melhor layout e espaçamento
# fig = make_subplots(
#     rows=2, cols=2,  
#     subplot_titles=(
#         'Internações por Mês', 
#         'Procedimentos com Maior Valor Médio',
#         'Tempo Médio de Internação', 
#         'Procedimentos Mais Comuns'
#     ),
#     vertical_spacing=0.15,  # Mais espaço entre os gráficos
#     horizontal_spacing=0.1,  # Melhor espaçamento lateral
#     specs=[[{"type": "bar"}, {"type": "bar"}],
#            [{"type": "bar"}, {"type": "bar"}]]
# )

# internacoes_mes = dados['SP_DTINTER'].dt.month.value_counts().sort_index()
# fig.add_trace(
#     go.Bar(
#         x=[calendar.month_name[m] for m in internacoes_mes.index], 
#         y=internacoes_mes.values,
#         marker_color='rgb(99, 110, 250)',  # Azul
#         name="Internações por Mês",
#         text=internacoes_mes.values,  # Exibir valores nas barras
#         textposition="outside",
#         hovertemplate='<b>Mês:</b> %{x}<br><b>Internações:</b> %{y}<extra></extra>'  # Tooltip para o mês
#     ),
#     row=1, col=1
# )

# valores_medios = dados.groupby('SP_N_PROC')['SP_VALATO'].mean().sort_values(ascending=False).head(10)
# # Criar uma coluna com nomes completos para tooltips
# nomes_completos_valores = valores_medios.index.tolist()
# fig.add_trace(
#     go.Bar(
#         x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in valores_medios.index],  # Truncar para 15 caracteres
#         y=valores_medios.values,
#         marker_color='rgb(239, 85, 59)',  # Vermelho
#         name="Maior Valor Médio",
#         text=[f"R$ {v:,.2f}" for v in valores_medios.values],  # Exibir valores formatados nas barras
#         textposition="outside",
#         hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Valor Médio (R$):</b> %{y:,.2f}<extra></extra>',  # Tooltip com nome completo
#         customdata=nomes_completos_valores  # Armazenar nomes completos para tooltips
#     ),
#     row=1, col=2
# )

# tempo_medio = dados.groupby('SP_N_PROC')['tempo_internacao'].mean().sort_values(ascending=False).head(10)
# # Criar uma coluna com nomes completos para tooltips
# nomes_completos_tempo = tempo_medio.index.tolist()
# fig.add_trace(
#     go.Bar(
#         x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in tempo_medio.index],  # Truncar para 15 caracteres
#         y=tempo_medio.values,
#         marker_color='rgb(0, 204, 150)',  # Verde
#         name="Tempo Médio de Internação",
#         text=[f"{v:.1f} dias" for v in tempo_medio.values],
#         textposition="outside",
#         hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Tempo Médio (dias):</b> %{y:.1f}<extra></extra>',  # Tooltip com nome completo
#         customdata=nomes_completos_tempo  # Armazenar nomes completos para tooltips
#     ),
#     row=2, col=1
# )

# top_proc = dados['SP_N_PROC'].value_counts().head(10)
# # Criar uma coluna com nomes completos para tooltips
# nomes_completos_proc = top_proc.index.tolist()
# fig.add_trace(
#     go.Bar(
#         x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in top_proc.index],  # Truncar para 15 caracteres
#         y=top_proc.values,
#         marker_color='rgb(171, 99, 250)',  # Roxo
#         name="Procedimentos Mais Comuns",
#         text=top_proc.values,
#         textposition="outside",
#         hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Ocorrências:</b> %{y}<extra></extra>',  # Tooltip com nome completo
#         customdata=nomes_completos_proc  # Armazenar nomes completos para tooltips
#     ),
#     row=2, col=2
# )
# fig.update_layout(
#     height=950,  
#     width=900,  
#     title_text="📊 Dashboard de Análise de Internações", 
#     title_x=0.5,  # Centralizar título
#     showlegend=False,  
#     margin=dict(l=80, r=50, t=100, b=80),
#     font=dict(family="Arial", size=12),  
#     template="plotly_white"  # Estilo mais moderno
# )

# fig.update_xaxes(
#     tickangle=30,  # Reduzir o ângulo para melhorar legibilidade
#     tickmode="array",
#     showticklabels=True
# )
# fig.update_yaxes(title_standoff=10)

# fig.show()
# fig.write_html("dashboard.html", auto_open=False)

# # 11. Salvar dados processados
# dados.to_csv('dados_RS.csv', index=False)

In [103]:
# import plotly.graph_objects as go
# import pandas as pd
# import calendar

# # 10.1. Gráfico de Internações por Mês
# internacoes_mes = dados['SP_DTINTER'].dt.month.value_counts().sort_index()
# fig_internacoes = go.Figure(
#     data=[go.Bar(
#         x=[calendar.month_name[m] for m in internacoes_mes.index], 
#         y=internacoes_mes.values,
#         marker_color='rgb(99, 110, 250)',  # Azul
#         name="Internações por Mês",
#         text=internacoes_mes.values,  # Exibir valores nas barras
#         textposition="outside",
#         hovertemplate='<b>Mês:</b> %{x}<br><b>Internações:</b> %{y}<extra></extra>'  # Tooltip para o mês
#     )]
# )

# fig_internacoes.update_layout(
#     height=400,  # Tamanho ajustado para caber na tela
#     width=900,   # Tamanho ajustado para caber na tela
#     title_text="Internações por Mês", 
#     title_x=0.5,  # Centralizar título
#     margin=dict(l=80, r=50, t=80, b=80),  # Margens menores
#     font=dict(family="Arial", size=12),  
#     template="plotly_white"  # Estilo mais moderno
# )

# fig_internacoes.update_xaxes(
#     tickangle=30,  # Reduzir o ângulo para melhorar legibilidade
#     tickmode="array",
#     showticklabels=True
# )
# fig_internacoes.update_yaxes(title_standoff=10)

# fig_internacoes.write_html("internacoes_mes.html", auto_open=False)

In [104]:
# # 10.2. Gráfico de Procedimentos com Maior Valor Médio
# valores_medios = dados.groupby('SP_N_PROC')['SP_VALATO'].mean().sort_values(ascending=False).head(10)
# nomes_completos_valores = valores_medios.index.tolist()
# fig_valores = go.Figure(
#     data=[go.Bar(
#         x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in valores_medios.index], 
#         y=valores_medios.values,
#         marker_color='rgb(239, 85, 59)',  # Vermelho
#         name="Maior Valor Médio",
#         text=[f"R$ {v:,.2f}" for v in valores_medios.values], 
#         textposition="outside",
#         hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Valor Médio (R$):</b> %{y:,.2f}<extra></extra>',
#         customdata=nomes_completos_valores  # Nomes completos para tooltips
#     )]
# )

# fig_valores.update_layout(
#     height=400,  # Tamanho ajustado para caber na tela
#     width=900,   # Tamanho ajustado para caber na tela
#     title_text="Procedimentos com Maior Valor Médio", 
#     title_x=0.5, 
#     margin=dict(l=80, r=50, t=80, b=80),  # Margens menores
#     font=dict(family="Arial", size=12),  
#     template="plotly_white" 
# )

# fig_valores.update_xaxes(
#     tickangle=30, 
#     tickmode="array",
#     showticklabels=True
# )
# fig_valores.update_yaxes(title_standoff=10)

# fig_valores.write_html("valores_medios.html", auto_open=False)

In [105]:
# # 10.3. Gráfico de Tempo Médio de Internação
# tempo_medio = dados.groupby('SP_N_PROC')['tempo_internacao'].mean().sort_values(ascending=False).head(10)
# nomes_completos_tempo = tempo_medio.index.tolist()
# fig_tempo = go.Figure(
#     data=[go.Bar(
#         x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in tempo_medio.index], 
#         y=tempo_medio.values,
#         marker_color='rgb(0, 204, 150)',  # Verde
#         name="Tempo Médio de Internação",
#         text=[f"{v:.1f} dias" for v in tempo_medio.values],
#         textposition="outside",
#         hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Tempo Médio (dias):</b> %{y:.1f}<extra></extra>',
#         customdata=nomes_completos_tempo  # Nomes completos para tooltips
#     )]
# )

# fig_tempo.update_layout(
#     height=400,  # Tamanho ajustado para caber na tela
#     width=900,   # Tamanho ajustado para caber na tela
#     title_text="Tempo Médio de Internação", 
#     title_x=0.5, 
#     margin=dict(l=80, r=50, t=80, b=80),  # Margens menores
#     font=dict(family="Arial", size=12),  
#     template="plotly_white" 
# )

# fig_tempo.update_xaxes(
#     tickangle=30, 
#     tickmode="array",
#     showticklabels=True
# )
# fig_tempo.update_yaxes(title_standoff=10)

# fig_tempo.write_html("tempo_medio.html", auto_open=False)

In [106]:
# # 10.4. Gráfico de Procedimentos Mais Comuns
# top_proc = dados['SP_N_PROC'].value_counts().head(10)
# nomes_completos_proc = top_proc.index.tolist()
# fig_comuns = go.Figure(
#     data=[go.Bar(
#         x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in top_proc.index], 
#         y=top_proc.values,
#         marker_color='rgb(171, 99, 250)',  # Roxo
#         name="Procedimentos Mais Comuns",
#         text=top_proc.values,
#         textposition="outside",
#         hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Ocorrências:</b> %{y}<extra></extra>',
#         customdata=nomes_completos_proc  # Nomes completos para tooltips
#     )]
# )

# fig_comuns.update_layout(
#     height=400,  # Tamanho ajustado para caber na tela
#     width=900,   # Tamanho ajustado para caber na tela
#     title_text="Procedimentos Mais Comuns", 
#     title_x=0.5, 
#     margin=dict(l=80, r=50, t=80, b=80),  # Margens menores
#     font=dict(family="Arial", size=12),  
#     template="plotly_white" 
# )

# fig_comuns.update_xaxes(
#     tickangle=30, 
#     tickmode="array",
#     showticklabels=True
# )
# fig_comuns.update_yaxes(title_standoff=10)

# fig_comuns.write_html("procedimentos_comuns.html", auto_open=False)

# # 11. Salvar dados processados
# #dados.to_csv('dados_RS.csv', index=False)

In [107]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
import calendar

# Criando subplots
fig = sp.make_subplots(
    rows=2, cols=2, 
    subplot_titles=[
        "Internações por Mês", 
        "Procedimentos Mais Comuns", 
        "Procedimentos com Maior Valor Médio", 
        "Internações por Dia da Semana"
    ]
)

# 1. Gráfico de Internações por Mês
internacoes_mes = dados['SP_DTINTER'].dt.month.value_counts().sort_index()
fig.add_trace(
    go.Bar(
        x=[calendar.month_name[m] for m in internacoes_mes.index], 
        y=internacoes_mes.values,
        marker_color='rgb(99, 110, 250)',  # Azul
        name="Internações por Mês",
        text=internacoes_mes.values,
        textposition="outside",
        hovertemplate='<b>Mês:</b> %{x}<br><b>Internações:</b> %{y}<extra></extra>'
    ),
    row=1, col=1
)

# 2. Gráfico de Procedimentos Mais Comuns
top_proc = dados['SP_N_PROC'].value_counts().head(10)
nomes_completos_proc = top_proc.index.tolist()
fig.add_trace(
    go.Bar(
        x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in top_proc.index], 
        y=top_proc.values,
        marker_color='rgb(171, 99, 250)',  # Roxo
        name="Procedimentos Mais Comuns",
        text=top_proc.values,
        textposition="outside",
        hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Ocorrências:</b> %{y}<extra></extra>',
        customdata=nomes_completos_proc  
    ),
    row=1, col=2
)

# 3. Gráfico de Procedimentos com Maior Valor Médio
valores_medios = dados.groupby('SP_N_PROC')['SP_VALATO'].mean().sort_values(ascending=False).head(10)
nomes_completos_valores = valores_medios.index.tolist()
fig.add_trace(
    go.Bar(
        x=[proc[:15] + '...' if len(proc) > 15 else proc for proc in valores_medios.index], 
        y=valores_medios.values,
        marker_color='rgb(239, 85, 59)',  # Vermelho
        name="Maior Valor Médio",
        text=[f"R$ {v:,.2f}" for v in valores_medios.values], 
        textposition="outside",
        hovertemplate='<b>Procedimento:</b> %{customdata}<br><b>Valor Médio (R$):</b> %{y:,.2f}<extra></extra>',
        customdata=nomes_completos_valores
    ),
    row=2, col=1
)

# 4. Gráfico de Internações por Dia da Semana (Com Cores Diferentes)
dados['dia_semana'] = dados['SP_DTINTER'].dt.day_name()
ordem_dias = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
nomes_dias_pt = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]

internacoes_dia = dados['dia_semana'].value_counts()
internacoes_dia = internacoes_dia.reindex(ordem_dias)  # Garantir ordem correta

# Definindo cores personalizadas para cada dia
cores_dias = {
    "Monday": "rgb(99, 110, 250)",    # Azul
    "Tuesday": "rgb(239, 85, 59)",    # Vermelho
    "Wednesday": "rgb(0, 204, 150)",  # Verde
    "Thursday": "rgb(171, 99, 250)",  # Roxo
    "Friday": "rgb(255, 161, 90)",    # Laranja
    "Saturday": "rgb(222, 51, 225)",  # Rosa
    "Sunday": "rgb(45, 186, 223)"     # Ciano
}

fig.add_trace(
    go.Bar(
        x=nomes_dias_pt, 
        y=internacoes_dia.values,
        marker=dict(color=[cores_dias[d] for d in ordem_dias]),  # Aplicando cores
        name="Internações por Dia da Semana",
        text=internacoes_dia.values,
        textposition="outside",
        hovertemplate='<b>Dia:</b> %{x}<br><b>Internações:</b> %{y}<extra></extra>'
    ),
    row=2, col=2
)

# Ajustando o layout
fig.update_layout(
    height=800, width=1200, 
    title_text="Análises de Internações e Procedimentos", 
    title_x=0.5,
    margin=dict(l=50, r=50, t=80, b=50),
    font=dict(family="Arial", size=12),
    template="plotly_white",
    showlegend=False
)

# Ajustando eixos
fig.update_yaxes(title_text="Quantidade de Internações", row=2, col=2)
fig.update_xaxes(title_standoff=10)

# Exportar HTML
fig.write_html("analise_completa.html", auto_open=False)
